In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import matplotlib.pylab as pylab
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei'] 

In [2]:
#导入训练数据
train_df = pd.read_csv('../../data/round1_ijcai_18_train_20180301.txt', sep=' ')
print(train_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478138 entries, 0 to 478137
Data columns (total 27 columns):
instance_id                  478138 non-null int64
item_id                      478138 non-null int64
item_category_list           478138 non-null object
item_property_list           478138 non-null object
item_brand_id                478138 non-null int64
item_city_id                 478138 non-null int64
item_price_level             478138 non-null int64
item_sales_level             478138 non-null int64
item_collected_level         478138 non-null int64
item_pv_level                478138 non-null int64
user_id                      478138 non-null int64
user_gender_id               478138 non-null int64
user_age_level               478138 non-null int64
user_occupation_id           478138 non-null int64
user_star_level              478138 non-null int64
context_id                   478138 non-null int64
context_timestamp            478138 non-null int64
context_page_id     

In [3]:
#尝试将时间戳转化为日期
# train_df['date'] = train_df.context_timestamp.map(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d %H:%M:%S"))
train_df['date'] = train_df.context_timestamp.map(lambda x: datetime.datetime.fromtimestamp(x))
train_df['weekday'] = train_df['date'].map(lambda x: x.weekday())
train_df['day'] = train_df['date'].map(lambda x: x.day)
train_df['hour'] = train_df['date'].map(lambda x: x.hour)
print(train_df[['date', 'context_timestamp', 'day', 'weekday']].head(10))

                 date  context_timestamp  day  weekday
0 2018-09-18 10:09:04         1537236544   18        1
1 2018-09-18 12:00:32         1537243232   18        1
2 2018-09-18 03:04:12         1537211052   18        1
3 2018-09-18 06:17:50         1537222670   18        1
4 2018-09-18 19:48:40         1537271320   18        1
5 2018-09-18 23:00:55         1537282855   18        1
6 2018-09-18 22:18:37         1537280317   18        1
7 2018-09-18 16:58:40         1537261120   18        1
8 2018-09-18 02:27:51         1537208871   18        1
9 2018-09-18 23:43:10         1537285390   18        1


In [4]:
#提取跟广告商品展示页面编号相关的特征
#第1页作为一个特征，2-5页作为一个特征，6-10页作为一个特征，10-20页作为一个特征
def getPageIdFuture(df):
    df['context_page_id_1'] = df['context_page_id'].map(lambda x: 1 if x == 4001 else 0)
    df['context_page_id_2'] = df['context_page_id'].map(lambda x: 1 if ((x < 4006) & (x > 4001)) else 0)
    df['context_page_id_3'] = df['context_page_id'].map(lambda x: 1 if ((x < 4011) & (x > 4005)) else 0)
    df['context_page_id_4'] = df['context_page_id'].map(lambda x: 1 if ((x <= 4020) & (x > 4010)) else 0)
    return df

train_df = getPageIdFuture(train_df)
print(train_df.head(10))

           instance_id              item_id  \
0   108641074714126964  3412720377098676069   
1  5754713551599725161  3412720377098676069   
2   842679481291040981  3412720377098676069   
3   937088850059189027  3412720377098676069   
4  7975697065017708072  3412720377098676069   
5  7764762765372067286  3412720377098676069   
6  6956333474094867789   285660928590172217   
7  8387099821892927911   285660928590172217   
8  4021878205550012615  5202355029344881809   
9  6499571365974135517   285660928590172217   

                        item_category_list  \
0  7908382889764677758;5799347067982556520   
1  7908382889764677758;5799347067982556520   
2  7908382889764677758;5799347067982556520   
3  7908382889764677758;5799347067982556520   
4  7908382889764677758;5799347067982556520   
5  7908382889764677758;5799347067982556520   
6  7908382889764677758;8277336076276184272   
7  7908382889764677758;8277336076276184272   
8  7908382889764677758;5755694407684602296   
9  7908382889764677758

In [5]:
#构造跟广告商品价格等级item_price_level相关的特征
#对于0-3或者9以上的作为一个特征，4-8作为一个特征
def getItemPriceLevelFuture(df):
    df['item_price_level_0'] = df['item_price_level'].map(lambda x: 1 if ((x >= 4) & (x <= 8)) else 0)
    df['item_price_level_1'] = df['item_price_level'].map(lambda x: 1 if ((x < 4) | (x > 8)) else 0)
    return df

train_df = getItemPriceLevelFuture(train_df)
print(train_df.head(10))

           instance_id              item_id  \
0   108641074714126964  3412720377098676069   
1  5754713551599725161  3412720377098676069   
2   842679481291040981  3412720377098676069   
3   937088850059189027  3412720377098676069   
4  7975697065017708072  3412720377098676069   
5  7764762765372067286  3412720377098676069   
6  6956333474094867789   285660928590172217   
7  8387099821892927911   285660928590172217   
8  4021878205550012615  5202355029344881809   
9  6499571365974135517   285660928590172217   

                        item_category_list  \
0  7908382889764677758;5799347067982556520   
1  7908382889764677758;5799347067982556520   
2  7908382889764677758;5799347067982556520   
3  7908382889764677758;5799347067982556520   
4  7908382889764677758;5799347067982556520   
5  7908382889764677758;5799347067982556520   
6  7908382889764677758;8277336076276184272   
7  7908382889764677758;8277336076276184272   
8  7908382889764677758;5755694407684602296   
9  7908382889764677758

In [6]:
#构造跟广告商品被展示次数等级item_pv_level相关的特征
#对于等级小于10的样本作为一个特征，其余的采用onehot处理
def getItemPvLevelFuture(df):
    df['item_pv_level_low'] = df['item_pv_level'].map(lambda x: 1 if ((x <= 14) | (x == 21)) else 0)
    df['item_pv_level_low'] = df['item_pv_level'].map(lambda x: 1 if ((x <= 20) | (x >= 15)) else 0)
    return df

train_df = getItemPvLevelFuture(train_df)
print(train_df.tail(10))

                instance_id              item_id  \
478128  3006683903772995202  7876305779987282497   
478129   810107468384208582   642074399208226630   
478130   440724230912710850  8714352873114629036   
478131  5307935891681475459  5040834653196495196   
478132   477737900698382237  5040834653196495196   
478133  5940763769799191887  5040834653196495196   
478134  3387284546470665526  5040834653196495196   
478135  5693770660150212848  5040834653196495196   
478136  4623253188146764341  5040834653196495196   
478137  8013124750038806831  3762437480819554366   

                             item_category_list  \
478128  7908382889764677758;5799347067982556520   
478129  7908382889764677758;2011981573061447208   
478130  7908382889764677758;8277336076276184272   
478131  7908382889764677758;5755694407684602296   
478132  7908382889764677758;5755694407684602296   
478133  7908382889764677758;5755694407684602296   
478134  7908382889764677758;5755694407684602296   
478135  79083828897

In [7]:
#构造跟店铺的评价数量等级相关的特征
#对于等级小于等于5或者大于等于22的作为一个特征，其余的先采用onehot编码处理
def getShopReviewNumLevelFuture(df):
    df['shop_review_num_level_lowHigh'] = df['shop_review_num_level'].map(lambda x: 1 if ((x <= 5) | (x >= 22)) else 0)
    return df

train_df = getShopReviewNumLevelFuture(train_df)
print(train_df.head(10))

           instance_id              item_id  \
0   108641074714126964  3412720377098676069   
1  5754713551599725161  3412720377098676069   
2   842679481291040981  3412720377098676069   
3   937088850059189027  3412720377098676069   
4  7975697065017708072  3412720377098676069   
5  7764762765372067286  3412720377098676069   
6  6956333474094867789   285660928590172217   
7  8387099821892927911   285660928590172217   
8  4021878205550012615  5202355029344881809   
9  6499571365974135517   285660928590172217   

                        item_category_list  \
0  7908382889764677758;5799347067982556520   
1  7908382889764677758;5799347067982556520   
2  7908382889764677758;5799347067982556520   
3  7908382889764677758;5799347067982556520   
4  7908382889764677758;5799347067982556520   
5  7908382889764677758;5799347067982556520   
6  7908382889764677758;8277336076276184272   
7  7908382889764677758;8277336076276184272   
8  7908382889764677758;5755694407684602296   
9  7908382889764677758

In [8]:
#构造跟店铺星级编号相关的特征
#由于存在缺失值，将缺失值作为一个种类，然后，将等级小于等于5008或者大于等于5019的作为一个种类，其余的作为一个种类
def getShopStarLevelFuture(df):
    df['shop_star_level_lowHigh'] = df['shop_star_level'].map(lambda x: 1 if (((x >= 4999) & (x <= 5008)) | (x >=5019)) else 0)
    df['shop_star_level_middle'] = df['shop_star_level'].map(lambda x: 1 if ((x > 5008) & (x < 5019)) else 0)
    df['shop_star_level_loss'] = df['shop_star_level'].map(lambda x: 1 if x == -1 else 0)
    return df

train_df = getShopStarLevelFuture(train_df)
print(train_df.head(10))

           instance_id              item_id  \
0   108641074714126964  3412720377098676069   
1  5754713551599725161  3412720377098676069   
2   842679481291040981  3412720377098676069   
3   937088850059189027  3412720377098676069   
4  7975697065017708072  3412720377098676069   
5  7764762765372067286  3412720377098676069   
6  6956333474094867789   285660928590172217   
7  8387099821892927911   285660928590172217   
8  4021878205550012615  5202355029344881809   
9  6499571365974135517   285660928590172217   

                        item_category_list  \
0  7908382889764677758;5799347067982556520   
1  7908382889764677758;5799347067982556520   
2  7908382889764677758;5799347067982556520   
3  7908382889764677758;5799347067982556520   
4  7908382889764677758;5799347067982556520   
5  7908382889764677758;5799347067982556520   
6  7908382889764677758;8277336076276184272   
7  7908382889764677758;8277336076276184272   
8  7908382889764677758;5755694407684602296   
9  7908382889764677758

In [9]:
#添加跟一天内时间点划分相关的数据，9-18,19-21,22-0,1-8
def getDayHourFuture(df):
    df['day_hour'] = 0
    df['day_hour'][((df.hour <= 18) & (df.hour >= 9))] = 1
    df['day_hour'][((df.hour <= 21) & (df.hour >= 19))] = 2
    df['day_hour'][((df.hour == 23) | (df.hour == 22) | (df.hour == 0))] = 3
    return df

train_df = getDayHourFuture(train_df)
print(train_df[['hour', 'day_hour']].head(10))

   hour  day_hour
0    10         1
1    12         1
2     3         0
3     6         0
4    19         2
5    23         3
6    22         3
7    16         1
8     2         0
9    23         3


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
def addSet(setList):
    allSet = []
    for member in setList:
        allSet = allSet + member
    allSet = set(allSet)
    return allSet

#处理跟商品类目和属性相关的特征
def getCategoryFuture(df):
    df['predict_category_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else list(kv.split(':')[0] for kv in str(x).split(';')))
    df['predict_category_set'] = df['predict_category_list'].map(lambda x: set(x))
    df['real_item_category_list'] = df['item_category_list'].map(lambda x: set(x.split(';')))
    
    df['predict_property_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else ((kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) if len(kv.split(':')) >= 2 else [] for kv in str(x).split(';')))
    df['predict_property_list'] = df['predict_property_list'].map(lambda x: addSet(x))
    df['item_property_list'] = df['item_property_list'].map(lambda x: set(x.split(';')))
    return df

train_df = getCategoryFuture(train_df)
print(train_df['predict_category_list'].head(10))
print(train_df['predict_property_list'].head(10))
print(train_df['real_item_category_list'].head(10))
print(train_df['item_property_list'].head(10))

0    [5799347067982556520, 509660095530134768, 5755...
1           [5799347067982556520, 7908382889764677758]
2    [5799347067982556520, 7258015885215914736, 790...
3    [509660095530134768, 5799347067982556520, 7908...
4           [5799347067982556520, 7908382889764677758]
5    [5799347067982556520, 8710739180200009128, 575...
6    [2011981573061447208, 8277336076276184272, 575...
7    [2011981573061447208, 8277336076276184272, 575...
8    [5755694407684602296, 5799347067982556520, 790...
9    [8277336076276184272, 509660095530134768, 5799...
Name: predict_category_list, dtype: object
0                                {9148482949976129397}
1    {9148482949976129397, 5195139481388729954, 917...
2                                {5131280576272319091}
3    {6165347051143749031, 9148482949976129397, 360...
4                                {9172976955054793469}
5           {9172976955054793469, 5131280576272319091}
6                                {7199361004668592209}
7                     

In [11]:
def getMatchProportion(df):
    match_category_proportion = []
    match_property_proportion = []
    for x,y,m,n in df[['real_item_category_list', 'predict_category_set', 'item_property_list', 'predict_property_list']].values:
        match_category = x & y
        match_property = m & n
        if len(y) > 0:
            category_proportion = len(match_category) / len(y)
            match_category_proportion.append(category_proportion)
        else:
            match_category_proportion.append(0)
        if len(n) > 0:
            property_proportion = len(match_property) / len(n)
            match_property_proportion.append(property_proportion)
        else:
            match_property_proportion.append(0)
    df['match_category_proportion'] = match_category_proportion
    df['match_property_proportion'] = match_property_proportion
    return df

train_df = getMatchProportion(train_df)
print(train_df[['match_category_proportion', 'match_property_proportion']].head(10))


   match_category_proportion  match_property_proportion
0                   0.400000                   0.000000
1                   1.000000                   0.200000
2                   0.666667                   1.000000
3                   0.400000                   0.000000
4                   1.000000                   1.000000
5                   0.400000                   1.000000
6                   0.333333                   1.000000
7                   0.333333                   1.000000
8                   0.333333                   0.000000
9                   0.333333                   0.666667


In [12]:
#构造跟预测数目相关的特征
def getPredictNumber(df):
    df['predict_category_number'] = df['predict_category_set'].map(lambda x: len(x))
    df['predict_property_number'] = df['predict_property_list'].map(lambda x: len(x))
    return df

train_df = getPredictNumber(train_df)
print(train_df[['predict_category_number', 'predict_property_number', 'is_trade']].head(10))

   predict_category_number  predict_property_number  is_trade
0                        5                        1         0
1                        2                        5         0
2                        3                        1         0
3                        5                        8         0
4                        2                        1         0
5                        5                        2         0
6                        6                        1         0
7                        6                        1         0
8                        6                        1         0
9                        6                        3         0


In [13]:
#构造跟类目预测精确性相关的特征
def getPredictAccuracy(df):
    isFirstCategoryIn = []
    isLastCategoryIn = []
    for x,y in df[['real_item_category_list', 'predict_category_list']].values:
        if y[0] in x:
            isFirstCategoryIn.append(1)
        else:
            isFirstCategoryIn.append(0)
        if y[len(y)-1] in x:
            isLastCategoryIn.append(1)
        else:
            isLastCategoryIn.append(0)
    df['isFirstCategoryIn'] = isFirstCategoryIn
    df['isLastCategoryIn'] = isLastCategoryIn
    return df

train_df = getPredictAccuracy(train_df)
print(train_df[['isFirstCategoryIn', 'isLastCategoryIn', 'is_trade']].head(10))

   isFirstCategoryIn  isLastCategoryIn  is_trade
0                  1                 1         0
1                  1                 1         0
2                  1                 1         0
3                  0                 0         0
4                  1                 1         0
5                  1                 1         0
6                  0                 0         0
7                  0                 0         0
8                  1                 0         0
9                  1                 1         0


In [14]:
# train_df['difference'] = train_df['lastOneHour_sameItem_count'] - train_df['lastOneHour_sameShop_count']
# print(len(train_df['instance_id'][train_df.difference != 0]))

In [15]:
#添加商品属性个数以及类目个数特征
def getCPNumber(df):
    df['category_number'] = df['item_category_list'].map(lambda x: len(x.split(';')))
    df['property_number'] = df['item_property_list'].map(lambda x: len(x))
    return df

train_df = getCPNumber(train_df)
print(train_df[['item_property_list', 'property_number', 'category_number']].head(10))

                                  item_property_list  property_number  \
0  {720840888466250585, 7270669313837600482, 8838...               22   
1  {720840888466250585, 7270669313837600482, 8838...               22   
2  {720840888466250585, 7270669313837600482, 8838...               22   
3  {720840888466250585, 7270669313837600482, 8838...               22   
4  {720840888466250585, 7270669313837600482, 8838...               22   
5  {720840888466250585, 7270669313837600482, 8838...               22   
6  {615076485672811995, 3802510553218572927, 2559...               50   
7  {615076485672811995, 3802510553218572927, 2559...               50   
8  {1665611025031010859, 5119621442439839441, 207...               46   
9  {615076485672811995, 3802510553218572927, 2559...               50   

   category_number  
0                2  
1                2  
2                2  
3                2  
4                2  
5                2  
6                2  
7                2  
8      

In [16]:
#首先获取转正率大于0.1以及转正样本数大于50的品牌集合
train_df_positive_brand = train_df[['item_brand_id']][train_df.is_trade == 1].drop_duplicates()
train_df_positive_brand['positive_counts'] = train_df_positive_brand['item_brand_id'].map(lambda x: len(train_df['item_brand_id'][(train_df.is_trade == 1) & (train_df.item_brand_id == x)])) 
train_df_positive_brand['all_counts'] = train_df_positive_brand['item_brand_id'].map(lambda x: len(train_df['item_brand_id'][(train_df.item_brand_id == x)])) 
train_df_positive_brand['proportion'] = train_df_positive_brand['positive_counts'] / train_df_positive_brand['all_counts']
train_df_positive_brand['proportion_label'] = train_df_positive_brand.proportion.map(lambda x: 0 if math.isnan(x) else int(x * 10))
proportion_brandId_set = set(train_df_positive_brand['item_brand_id'][train_df_positive_brand.proportion > 0.1])
number_brandId_set = set(train_df_positive_brand['item_brand_id'][train_df_positive_brand.positive_counts > 50])
print(len(proportion_brandId_set))
print(len(number_brandId_set))

#处理跟品牌相关的特征，转正率大于0.1的作为一个特征，转正样本数大于50的也作为一个特征
def getBrandFuture(df, proportion_brandId_set, number_brandId_set):
    df['is_highProportion_brand'] = df['item_brand_id'].map(lambda x: 1 if x in proportion_brandId_set else 0)
    df['is_highSale_brand'] = df['item_brand_id'].map(lambda x: 1 if x in number_brandId_set else 0)
    return df

train_df = getBrandFuture(train_df, proportion_brandId_set, number_brandId_set)
print(train_df[['item_brand_id', 'is_highProportion_brand', 'is_highSale_brand']][train_df.is_trade == 1].head(30))

72
24
            item_brand_id  is_highProportion_brand  is_highSale_brand
14    9057103201734987852                        0                  0
52    7066302540842412840                        0                  1
55    7066302540842412840                        0                  1
83    7066302540842412840                        0                  1
129   7066302540842412840                        0                  1
271   7066302540842412840                        0                  1
816   7066302540842412840                        0                  1
1068  7066302540842412840                        0                  1
1109  7066302540842412840                        0                  1
1576  7066302540842412840                        0                  1
1837  7066302540842412840                        0                  1
1980   605266119463963358                        0                  0
1996   605266119463963358                        0                  0
2017   6052661

In [17]:
#获取组合特征函数
def zuheFuture(df, colName1, colName2, newColName):
    df[newColName] = df[colName1].astype(str) + df[colName2].astype(str)
    df[newColName] = df[newColName].astype(int)
    return df

#尝试获取跟广告商品相关的组合特征
def getItemZuheFuture(df):
    df = zuheFuture(df, 'item_sales_level', 'item_price_level', 'item_sales_price')
    df = zuheFuture(df, 'item_sales_level', 'item_collected_level', 'item_sales_collected')
    df = zuheFuture(df, 'item_sales_level', 'item_pv_level', 'item_sales_pv')
    df = zuheFuture(df, 'item_price_level', 'item_collected_level', 'item_price_collected')
    df = zuheFuture(df, 'item_price_level', 'item_pv_level', 'item_price_pv')
    df = zuheFuture(df, 'item_collected_level', 'item_pv_level', 'item_collected_pv')
    return df

train_df = getItemZuheFuture(train_df)
print(train_df[['item_sales_level', 'item_price_level', 'item_sales_price']][train_df.item_sales_price < 0].head(10))


       item_sales_level  item_price_level  item_sales_price
11498                -1                 7               -17
11559                -1                 7               -17
11565                -1                 7               -17
11677                -1                 5               -15
12838                -1                 5               -15
12842                -1                 5               -15
12846                -1                 5               -15
12859                -1                 5               -15
12877                -1                 5               -15
12887                -1                 5               -15


In [18]:
#尝试获取跟用户相关的组合特征
def getUserZuheFuture(df):
    temp = df.copy()
    for col in ['user_age_level', 'user_occupation_id', 'user_star_level']:
        temp[col] = temp[col].map(lambda x: 0 if x == -1 else x)
    
    temp = zuheFuture(temp, 'user_gender_id', 'user_age_level', 'user_gender_age')
    temp = zuheFuture(temp, 'user_gender_id', 'user_occupation_id', 'user_gender_occupation')
    temp = zuheFuture(temp, 'user_gender_id', 'user_star_level', 'user_gender_star')
    temp = zuheFuture(temp, 'user_age_level', 'user_occupation_id', 'user_age_occupation')
    temp = zuheFuture(temp, 'user_age_level', 'user_star_level', 'user_age_star')
    temp = zuheFuture(temp, 'user_occupation_id', 'user_star_level', 'user_occupation_star')
    
    for col in ['user_gender_age', 'user_gender_occupation', 'user_gender_star', 'user_age_occupation', 'user_age_star', 'user_occupation_star']:
        df[col] = temp[col]
    
    return df

train_df = getUserZuheFuture(train_df)
print(train_df[['user_gender_id', 'user_age_level', 'user_gender_age']].head(10))


   user_gender_id  user_age_level  user_gender_age
0               1            1003            11003
1               0            1002             1002
2               0            1003             1003
3               1            1004            11004
4               0            1002             1002
5               1            1004            11004
6               1            1006            11006
7               0            1002             1002
8               0            1003             1003
9               0            1003             1003


In [19]:
#尝试获取跟店铺相关的组合特征
def getShopZuheFuture(df):
    df = zuheFuture(df, 'shop_review_num_level', 'shop_star_level', 'shop_review_star')
    return df

train_df = getShopZuheFuture(train_df)
print(train_df[['shop_review_num_level', 'shop_star_level', 'shop_review_star']].head(10))


   shop_review_num_level  shop_star_level  shop_review_star
0                      4             5002             45002
1                      4             5002             45002
2                      4             5002             45002
3                      4             5002             45002
4                      4             5002             45002
5                      4             5002             45002
6                     15             5012            155012
7                     15             5012            155012
8                     15             5012            155012
9                     15             5012            155012


In [20]:
#导入初赛测试集数据
test_df_a = pd.read_csv('../../data/round1_ijcai_18_test_a_20180301.txt', sep=' ')
test_df_b = pd.read_csv('../../data/round1_ijcai_18_test_b_20180418.txt', sep=' ')

test_df = pd.concat([test_df_a, test_df_b], ignore_index=True)

print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61259 entries, 0 to 61258
Data columns (total 26 columns):
instance_id                  61259 non-null int64
item_id                      61259 non-null int64
item_category_list           61259 non-null object
item_property_list           61259 non-null object
item_brand_id                61259 non-null int64
item_city_id                 61259 non-null int64
item_price_level             61259 non-null int64
item_sales_level             61259 non-null int64
item_collected_level         61259 non-null int64
item_pv_level                61259 non-null int64
user_id                      61259 non-null int64
user_gender_id               61259 non-null int64
user_age_level               61259 non-null int64
user_occupation_id           61259 non-null int64
user_star_level              61259 non-null int64
context_id                   61259 non-null int64
context_timestamp            61259 non-null int64
context_page_id              61259 non-

In [21]:
#对测试集和训练集采取相同的处理
test_df['date'] = test_df.context_timestamp.map(lambda x: datetime.datetime.fromtimestamp(x))
test_df['weekday'] = test_df['date'].map(lambda x: x.weekday())
test_df['day'] = test_df['date'].map(lambda x: x.day)
test_df['hour'] = test_df['date'].map(lambda x: x.hour)
test_df = getDayHourFuture(test_df)

test_df = getPageIdFuture(test_df)

test_df = getItemPriceLevelFuture(test_df)

test_df = getItemPvLevelFuture(test_df)

test_df = getShopReviewNumLevelFuture(test_df)

test_df = getShopStarLevelFuture(test_df)

test_df = getCategoryFuture(test_df)
test_df = getMatchProportion(test_df)
test_df = getPredictNumber(test_df)
test_df = getPredictAccuracy(test_df)

test_df['real_first_category'] = test_df['item_category_list'].map(lambda x: np.nan if len(x.split(';')) < 1 else x.split(';')[0])
test_df['real_last_category'] = test_df['item_category_list'].map(lambda x: np.nan if len(x.split(';')) < 1 else x[len(x.split(';')) -1])

test_df = getBrandFuture(test_df, proportion_brandId_set, number_brandId_set)

test_df = getCPNumber(test_df)

test_df = getItemZuheFuture(test_df)
test_df = getUserZuheFuture(test_df)
test_df = getShopZuheFuture(test_df)

print(test_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


           instance_id              item_id  \
0  2475218615076601065  2275895163219263378   
1   398316874173557226  7096238490711246967   
2  6586402638209028583  7096238490711246967   
3  1040996105851528465  7096238490711246967   
4  6316278569655873454  7096238490711246967   
5   868158305045921978  7096238490711246967   
6  5713520501786699854  1171140458228333440   
7   932945015407923184  7020520887593189887   
8  1919197847086752313  8143258757857432437   
9   304887065966615346  8143258757857432437   

                        item_category_list  \
0  7908382889764677758;8277336076276184272   
1  7908382889764677758;5755694407684602296   
2  7908382889764677758;5755694407684602296   
3  7908382889764677758;5755694407684602296   
4  7908382889764677758;5755694407684602296   
5  7908382889764677758;5755694407684602296   
6  7908382889764677758;4879721024980945592   
7  7908382889764677758;2011981573061447208   
8  7908382889764677758;2011981573061447208   
9  7908382889764677758

In [22]:
# 统计过去一个小时某用户点击某个相同商品的次数，同时对训练集和测试集进行处理
def getOneHourSameItemCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item'] = tempDf['user_item'].shift(1)
    tempDf['last_user_item'] = tempDf['last_user_item']==tempDf['user_item']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameItem_count'] = hourShowList
    temp = pd.merge(temp_df[['user_item', 'date']], tempDf[['user_item', 'date', 'lastOneHour_sameItem_count']], on = ['user_item', 'date'], how = 'left')
#     print(temp_df[['user_item', 'date']].head())
#     print(tempDf[['user_item', 'date', 'lastOneHour_sameItem_count']][tempDf.user_item == '15383240786656120175586531724608162781'].head())
#     print(temp[temp.lastOneHour_sameItem_count > 0].head())
    train_df['lastOneHour_sameItem_count'] = temp['lastOneHour_sameItem_count'][:len(train_df['user_id'])]
    test_df['lastOneHour_sameItem_count'] = temp['lastOneHour_sameItem_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneHourSameItemCount(train_df, test_df)
print(train_df[['date', 'lastOneHour_sameItem_count']].head(10))

                 date  lastOneHour_sameItem_count
0 2018-09-18 10:09:04                         0.0
1 2018-09-18 12:00:32                         0.0
2 2018-09-18 03:04:12                         0.0
3 2018-09-18 06:17:50                         0.0
4 2018-09-18 19:48:40                         0.0
5 2018-09-18 23:00:55                         0.0
6 2018-09-18 22:18:37                         0.0
7 2018-09-18 16:58:40                         0.0
8 2018-09-18 02:27:51                         0.0
9 2018-09-18 23:43:10                         0.0


In [23]:
#构造跟商品实际的根类目和叶子类目特征
train_df['real_first_category'] = train_df['item_category_list'].map(lambda x: np.nan if len(x.split(';')) < 1 else x.split(';')[0])
train_df['real_last_category'] = train_df['item_category_list'].map(lambda x: np.nan if len(x.split(';')) < 1 else x[len(x.split(';')) -1])

# 统计过去一个小时某用户点击同种根类目商品的次数，同时对训练集和测试集进行处理
def getOneHourSameFirstCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_first_category', 'date', 'instance_id']], test_df[['user_id', 'real_first_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_first_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_first_category_str'] = temp_df['real_first_category'].map(lambda x: str(x))
    temp_df['user_real_first_category'] = temp_df['user_id_str'] + temp_df['real_first_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_first_category','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameFirstCategory_count'] = hourShowList
    temp = pd.merge(temp_df[['user_real_first_category', 'date']], tempDf[['user_real_first_category', 'date', 'lastOneHour_sameFirstCategory_count']], on = ['user_real_first_category', 'date'], how = 'left')
    train_df['lastOneHour_sameFirstCategory_count'] = temp['lastOneHour_sameFirstCategory_count'][:len(train_df['user_id'])]
    test_df['lastOneHour_sameFirstCategory_count'] = temp['lastOneHour_sameFirstCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneHourSameFirstCategoryCount(train_df, test_df)
print(train_df[['date', 'lastOneHour_sameFirstCategory_count']].head(10))

                 date  lastOneHour_sameFirstCategory_count
0 2018-09-18 10:09:04                                  0.0
1 2018-09-18 12:00:32                                  1.0
2 2018-09-18 03:04:12                                  0.0
3 2018-09-18 06:17:50                                  0.0
4 2018-09-18 19:48:40                                  0.0
5 2018-09-18 23:00:55                                  1.0
6 2018-09-18 22:18:37                                  7.0
7 2018-09-18 16:58:40                                  0.0
8 2018-09-18 02:27:51                                  0.0
9 2018-09-18 23:43:10                                  3.0


In [24]:
# 统计过去一个小时某用户点击同种叶子类目商品的次数，同时对训练集和测试集进行处理
def getOneHourSameLastCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_last_category', 'date', 'instance_id']], test_df[['user_id', 'real_last_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_last_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_last_category_str'] = temp_df['real_last_category'].map(lambda x: str(x))
    temp_df['user_real_last_category'] = temp_df['user_id_str'] + temp_df['real_last_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_last_category','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameLastCategory_count'] = hourShowList
    temp = pd.merge(temp_df[['user_real_last_category', 'date']], tempDf[['user_real_last_category', 'date', 'lastOneHour_sameLastCategory_count']], on = ['user_real_last_category', 'date'], how = 'left')
    train_df['lastOneHour_sameLastCategory_count'] = temp['lastOneHour_sameLastCategory_count'][:len(train_df['user_id'])]
    test_df['lastOneHour_sameLastCategory_count'] = temp['lastOneHour_sameLastCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneHourSameLastCategoryCount(train_df, test_df)
print(train_df[['date', 'lastOneHour_sameLastCategory_count']].head(10))

                 date  lastOneHour_sameLastCategory_count
0 2018-09-18 10:09:04                                 0.0
1 2018-09-18 12:00:32                                 1.0
2 2018-09-18 03:04:12                                 0.0
3 2018-09-18 06:17:50                                 0.0
4 2018-09-18 19:48:40                                 0.0
5 2018-09-18 23:00:55                                 1.0
6 2018-09-18 22:18:37                                 7.0
7 2018-09-18 16:58:40                                 0.0
8 2018-09-18 02:27:51                                 0.0
9 2018-09-18 23:43:10                                 3.0


In [25]:
# 统计过去一个小时某用户点击同种品牌商品的次数，同时对训练集和测试集进行处理
def getOneHourSameBrandCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_brand_id', 'date', 'instance_id']], test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['brand_id_str'] = temp_df['item_brand_id'].map(lambda x: str(x))
    temp_df['user_brand_id'] = temp_df['user_id_str'] + temp_df['brand_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_brand_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_brand_id'] = tempDf['user_brand_id'].shift(1)
    tempDf['last_user_brand_id'] = tempDf['last_user_brand_id']==tempDf['user_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_brand_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_brand_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameBrand_count'] = hourShowList
    temp = pd.merge(temp_df[['user_brand_id', 'date']], tempDf[['user_brand_id', 'date', 'lastOneHour_sameBrand_count']], on = ['user_brand_id', 'date'], how = 'left')
    train_df['lastOneHour_sameBrand_count'] = temp['lastOneHour_sameBrand_count'][:len(train_df['user_id'])]
    test_df['lastOneHour_sameBrand_count'] = temp['lastOneHour_sameBrand_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneHourSameBrandCount(train_df, test_df)
print(train_df[['date', 'lastOneHour_sameBrand_count']].head(10))


                 date  lastOneHour_sameBrand_count
0 2018-09-18 10:09:04                          0.0
1 2018-09-18 12:00:32                          0.0
2 2018-09-18 03:04:12                          0.0
3 2018-09-18 06:17:50                          0.0
4 2018-09-18 19:48:40                          0.0
5 2018-09-18 23:00:55                          0.0
6 2018-09-18 22:18:37                          0.0
7 2018-09-18 16:58:40                          0.0
8 2018-09-18 02:27:51                          0.0
9 2018-09-18 23:43:10                          0.0


In [26]:
# 统计过去一个小时某用户点击同种店铺商品的次数，同时对训练集和测试集进行处理
def getOneHourSameShopCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'shop_id', 'date', 'instance_id']], test_df[['user_id', 'shop_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'shop_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_shop_id_str'] = temp_df['shop_id'].map(lambda x: str(x))
    temp_df['user_item_shop_id'] = temp_df['user_id_str'] + temp_df['item_shop_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_shop_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_shop_id'] = tempDf['user_item_shop_id'].shift(1)
    tempDf['last_user_item_shop_id'] = tempDf['last_user_item_shop_id']==tempDf['user_item_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_shop_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_shop_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameShop_count'] = hourShowList
    temp = pd.merge(temp_df[['user_item_shop_id', 'date']], tempDf[['user_item_shop_id', 'date', 'lastOneHour_sameShop_count']], on = ['user_item_shop_id', 'date'], how = 'left')
    train_df['lastOneHour_sameShop_count'] = temp['lastOneHour_sameShop_count'][:len(train_df['user_id'])]
    test_df['lastOneHour_sameShop_count'] = temp['lastOneHour_sameShop_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneHourSameShopCount(train_df, test_df)
print(train_df[['date', 'lastOneHour_sameShop_count']].head(10))


                 date  lastOneHour_sameShop_count
0 2018-09-18 10:09:04                         0.0
1 2018-09-18 12:00:32                         0.0
2 2018-09-18 03:04:12                         0.0
3 2018-09-18 06:17:50                         0.0
4 2018-09-18 19:48:40                         0.0
5 2018-09-18 23:00:55                         0.0
6 2018-09-18 22:18:37                         0.0
7 2018-09-18 16:58:40                         0.0
8 2018-09-18 02:27:51                         0.0
9 2018-09-18 23:43:10                         0.0


In [27]:
# 获取是否是该用户在这1个小时内第一次点击这个商品的特征，同时对训练集和测试集进行处理
def getIsOneHourFirstClickItem(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item_id'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            if len(hourShowTemp) > 0:
                hourShowList.append(0)
            else:
                hourShowList.append(1)
            hourShowTemp[dt] = show
        else:
            hourShowList.append(1)
            hourShowTemp = {dt:show}
    tempDf['isLastOneHour_firstClickItem'] = hourShowList
    temp = pd.merge(temp_df[['user_item_id', 'date']], tempDf[['user_item_id', 'date', 'isLastOneHour_firstClickItem']], on = ['user_item_id', 'date'], how = 'left')
#     print(temp[['user_item_id', 'date', 'isLastOneHour_firstClickItem']].head(10))
#     print(temp[['user_item_id', 'date', 'isLastOneHour_firstClickItem', 'is_trade']][temp.user_item_id == '62033080084805934235202355029344881809'].head(10))
#     print(len(temp[temp.isLastOneHour_firstClickItem == 0]))
#     print(len(temp[(temp.isLastOneHour_firstClickItem == 0) & (temp.is_trade == 1)]))
#     print(len(temp[temp.isLastOneHour_firstClickItem == 1]))
#     print(len(temp[(temp.isLastOneHour_firstClickItem == 1) & (temp.is_trade == 1)]))
    train_df['isLastOneHour_firstClickItem'] = temp['isLastOneHour_firstClickItem'][:len(train_df['user_id'])]
    test_df['isLastOneHour_firstClickItem'] = temp['isLastOneHour_firstClickItem'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsOneHourFirstClickItem(train_df, test_df)
print(train_df[['date', 'isLastOneHour_firstClickItem']].head(10))

                 date  isLastOneHour_firstClickItem
0 2018-09-18 10:09:04                             1
1 2018-09-18 12:00:32                             1
2 2018-09-18 03:04:12                             1
3 2018-09-18 06:17:50                             1
4 2018-09-18 19:48:40                             1
5 2018-09-18 23:00:55                             1
6 2018-09-18 22:18:37                             1
7 2018-09-18 16:58:40                             1
8 2018-09-18 02:27:51                             1
9 2018-09-18 23:43:10                             1


In [28]:
# 统计过去一天某用户点击某个相同商品的次数，同时对训练集和测试集进行处理
def getOneDaySameItemCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item'] = tempDf['user_item'].shift(1)
    tempDf['last_user_item'] = tempDf['last_user_item']==tempDf['user_item']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    dayShowList = []
    dayShowTemp = {}
    for same, dt, show in tempDf[['last_user_item','date','show']].values:
        if same:
            [dayShowTemp.pop(k) for k in list(dayShowTemp) if k<dt-timedelta(days=1)]
            dayShowList.append(np.sum(list(dayShowTemp.values())))
            dayShowTemp[dt] = show
        else:
            dayShowList.append(0)
            dayShowTemp = {dt:show}
    tempDf['lastOneDay_sameItem_count'] = dayShowList
    temp = pd.merge(temp_df[['user_item', 'date']], tempDf[['user_item', 'date', 'lastOneDay_sameItem_count']], on = ['user_item', 'date'], how = 'left')
    train_df['lastOneDay_sameItem_count'] = temp['lastOneDay_sameItem_count'][:len(train_df['user_id'])]
    test_df['lastOneDay_sameItem_count'] = temp['lastOneDay_sameItem_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneDaySameItemCount(train_df, test_df)
print(train_df[['date', 'lastOneDay_sameItem_count']].head(10))


                 date  lastOneDay_sameItem_count
0 2018-09-18 10:09:04                        0.0
1 2018-09-18 12:00:32                        0.0
2 2018-09-18 03:04:12                        0.0
3 2018-09-18 06:17:50                        0.0
4 2018-09-18 19:48:40                        0.0
5 2018-09-18 23:00:55                        0.0
6 2018-09-18 22:18:37                        0.0
7 2018-09-18 16:58:40                        0.0
8 2018-09-18 02:27:51                        0.0
9 2018-09-18 23:43:10                        0.0


In [29]:
# 统计过去一天某用户点击同种根类目商品的次数，同时对训练集和测试集进行处理
def getOneDaySameFirstCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_first_category', 'date', 'instance_id']], test_df[['user_id', 'real_first_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_first_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_first_category_str'] = temp_df['real_first_category'].map(lambda x: str(x))
    temp_df['user_real_first_category'] = temp_df['user_id_str'] + temp_df['real_first_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    dayShowList = []
    dayShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_first_category','date','show']].values:
        if same:
            [dayShowTemp.pop(k) for k in list(dayShowTemp) if k<dt-timedelta(days=1)]
            dayShowList.append(np.sum(list(dayShowTemp.values())))
            dayShowTemp[dt] = show
        else:
            dayShowList.append(0)
            dayShowTemp = {dt:show}
    tempDf['lastOneDay_sameFirstCategory_count'] = dayShowList
    temp = pd.merge(temp_df[['user_real_first_category', 'date']], tempDf[['user_real_first_category', 'date', 'lastOneDay_sameFirstCategory_count']], on = ['user_real_first_category', 'date'], how = 'left')
    train_df['lastOneDay_sameFirstCategory_count'] = temp['lastOneDay_sameFirstCategory_count'][:len(train_df['user_id'])]
    test_df['lastOneDay_sameFirstCategory_count'] = temp['lastOneDay_sameFirstCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneDaySameFirstCategoryCount(train_df, test_df)
print(train_df[['date', 'lastOneDay_sameFirstCategory_count']].head(10))


                 date  lastOneDay_sameFirstCategory_count
0 2018-09-18 10:09:04                                 0.0
1 2018-09-18 12:00:32                                 1.0
2 2018-09-18 03:04:12                                 0.0
3 2018-09-18 06:17:50                                 0.0
4 2018-09-18 19:48:40                                 0.0
5 2018-09-18 23:00:55                                 1.0
6 2018-09-18 22:18:37                                19.0
7 2018-09-18 16:58:40                                 0.0
8 2018-09-18 02:27:51                                 0.0
9 2018-09-18 23:43:10                                 3.0


In [30]:
# 统计过去一个天某用户点击同种叶子类目商品的次数，同时对训练集和测试集进行处理
def getOneDaySameLastCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_last_category', 'date', 'instance_id']], test_df[['user_id', 'real_last_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_last_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_last_category_str'] = temp_df['real_last_category'].map(lambda x: str(x))
    temp_df['user_real_last_category'] = temp_df['user_id_str'] + temp_df['real_last_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    dayShowList = []
    dayShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_last_category','date','show']].values:
        if same:
            [dayShowTemp.pop(k) for k in list(dayShowTemp) if k<dt-timedelta(days=1)]
            dayShowList.append(np.sum(list(dayShowTemp.values())))
            dayShowTemp[dt] = show
        else:
            dayShowList.append(0)
            dayShowTemp = {dt:show}
    tempDf['lastOneDay_sameLastCategory_count'] = dayShowList
    temp = pd.merge(temp_df[['user_real_last_category', 'date']], tempDf[['user_real_last_category', 'date', 'lastOneDay_sameLastCategory_count']], on = ['user_real_last_category', 'date'], how = 'left')
    train_df['lastOneDay_sameLastCategory_count'] = temp['lastOneDay_sameLastCategory_count'][:len(train_df['user_id'])]
    test_df['lastOneDay_sameLastCategory_count'] = temp['lastOneDay_sameLastCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneDaySameLastCategoryCount(train_df, test_df)
print(train_df[['date', 'lastOneDay_sameLastCategory_count']].head(10))


                 date  lastOneDay_sameLastCategory_count
0 2018-09-18 10:09:04                                0.0
1 2018-09-18 12:00:32                                1.0
2 2018-09-18 03:04:12                                0.0
3 2018-09-18 06:17:50                                0.0
4 2018-09-18 19:48:40                                0.0
5 2018-09-18 23:00:55                                1.0
6 2018-09-18 22:18:37                               19.0
7 2018-09-18 16:58:40                                0.0
8 2018-09-18 02:27:51                                0.0
9 2018-09-18 23:43:10                                3.0


In [31]:
# 统计过去一个天某用户点击同种品牌商品的次数，同时对训练集和测试集进行处理
def getOneDaySameBrandCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_brand_id', 'date', 'instance_id']], test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['brand_id_str'] = temp_df['item_brand_id'].map(lambda x: str(x))
    temp_df['user_brand_id'] = temp_df['user_id_str'] + temp_df['brand_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_brand_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_brand_id'] = tempDf['user_brand_id'].shift(1)
    tempDf['last_user_brand_id'] = tempDf['last_user_brand_id']==tempDf['user_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_brand_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    dayShowList = []
    dayShowTemp = {}
    for same, dt, show in tempDf[['last_user_brand_id','date','show']].values:
        if same:
            [dayShowTemp.pop(k) for k in list(dayShowTemp) if k<dt-timedelta(days=1)]
            dayShowList.append(np.sum(list(dayShowTemp.values())))
            dayShowTemp[dt] = show
        else:
            dayShowList.append(0)
            dayShowTemp = {dt:show}
    tempDf['lastOneDay_sameBrand_count'] = dayShowList
    temp = pd.merge(temp_df[['user_brand_id', 'date']], tempDf[['user_brand_id', 'date', 'lastOneDay_sameBrand_count']], on = ['user_brand_id', 'date'], how = 'left')
    train_df['lastOneDay_sameBrand_count'] = temp['lastOneDay_sameBrand_count'][:len(train_df['user_id'])]
    test_df['lastOneDay_sameBrand_count'] = temp['lastOneDay_sameBrand_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneDaySameBrandCount(train_df, test_df)
print(train_df[['date', 'lastOneDay_sameBrand_count']].head(10))


                 date  lastOneDay_sameBrand_count
0 2018-09-18 10:09:04                         0.0
1 2018-09-18 12:00:32                         0.0
2 2018-09-18 03:04:12                         0.0
3 2018-09-18 06:17:50                         0.0
4 2018-09-18 19:48:40                         0.0
5 2018-09-18 23:00:55                         0.0
6 2018-09-18 22:18:37                         0.0
7 2018-09-18 16:58:40                         0.0
8 2018-09-18 02:27:51                         0.0
9 2018-09-18 23:43:10                         0.0


In [32]:
# 统计过去一个天某用户点击同种店铺商品的次数，同时对训练集和测试集进行处理
def getOneDaySameShopCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'shop_id', 'date', 'instance_id']], test_df[['user_id', 'shop_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'shop_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_shop_id_str'] = temp_df['shop_id'].map(lambda x: str(x))
    temp_df['user_item_shop_id'] = temp_df['user_id_str'] + temp_df['item_shop_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_shop_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_shop_id'] = tempDf['user_item_shop_id'].shift(1)
    tempDf['last_user_item_shop_id'] = tempDf['last_user_item_shop_id']==tempDf['user_item_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_shop_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    dayShowList = []
    dayShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_shop_id','date','show']].values:
        if same:
            [dayShowTemp.pop(k) for k in list(dayShowTemp) if k<dt-timedelta(hours=1)]
            dayShowList.append(np.sum(list(dayShowTemp.values())))
            dayShowTemp[dt] = show
        else:
            dayShowList.append(0)
            dayShowTemp = {dt:show}
    tempDf['lastOneDay_sameShop_count'] = dayShowList
    temp = pd.merge(temp_df[['user_item_shop_id', 'date']], tempDf[['user_item_shop_id', 'date', 'lastOneDay_sameShop_count']], on = ['user_item_shop_id', 'date'], how = 'left')
    train_df['lastOneDay_sameShop_count'] = temp['lastOneDay_sameShop_count'][:len(train_df['user_id'])]
    test_df['lastOneDay_sameShop_count'] = temp['lastOneDay_sameShop_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getOneDaySameShopCount(train_df, test_df)
print(train_df[['date', 'lastOneDay_sameShop_count']].head(10))


                 date  lastOneDay_sameShop_count
0 2018-09-18 10:09:04                        0.0
1 2018-09-18 12:00:32                        0.0
2 2018-09-18 03:04:12                        0.0
3 2018-09-18 06:17:50                        0.0
4 2018-09-18 19:48:40                        0.0
5 2018-09-18 23:00:55                        0.0
6 2018-09-18 22:18:37                        0.0
7 2018-09-18 16:58:40                        0.0
8 2018-09-18 02:27:51                        0.0
9 2018-09-18 23:43:10                        0.0


In [33]:
# 统计历史记录中某用户点击某个相同商品的次数，同时对训练集和测试集进行处理
def getHistorySameItemCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item'] = tempDf['user_item'].shift(1)
    tempDf['last_user_item'] = tempDf['last_user_item']==tempDf['user_item']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    historyShowList = []
    historyShowTemp = {}
    for same, dt, show in tempDf[['last_user_item','date','show']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            historyShowList.append(np.sum(list(historyShowTemp.values())))
            historyShowTemp[dt] = show
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:show}
    tempDf['history_sameItem_count'] = historyShowList
    temp = pd.merge(temp_df[['user_item', 'date']], tempDf[['user_item', 'date', 'history_sameItem_count']], on = ['user_item', 'date'], how = 'left')
#     print(temp_df[['user_item', 'date']].head())
#     print(tempDf[['user_item', 'date', 'lastOneHour_sameItem_count']][tempDf.user_item == '15383240786656120175586531724608162781'].head())
#     print(temp[temp.lastOneHour_sameItem_count > 0].head())
    train_df['history_sameItem_count'] = temp['history_sameItem_count'][:len(train_df['user_id'])]
    test_df['history_sameItem_count'] = temp['history_sameItem_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getHistorySameItemCount(train_df, test_df)
print(train_df[['date', 'history_sameItem_count', 'lastOneHour_sameItem_count']][train_df.lastOneHour_sameItem_count > 0].head(10))

                   date  history_sameItem_count  lastOneHour_sameItem_count
60  2018-09-18 19:48:23                       1                         1.0
103 2018-09-18 14:35:06                       1                         1.0
129 2018-09-18 13:13:32                       1                         1.0
146 2018-09-18 09:28:18                       1                         1.0
156 2018-09-18 12:18:04                       1                         1.0
284 2018-09-18 14:32:35                       1                         1.0
293 2018-09-18 21:03:05                       3                         2.0
306 2018-09-18 16:22:59                       1                         1.0
436 2018-09-18 20:40:27                       1                         1.0
525 2018-09-18 11:09:58                       1                         1.0


In [34]:
# 统计历史记录中某用户点击某个相同根类目商品的次数，同时对训练集和测试集进行处理
def getHistorySameFirstCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_first_category', 'date', 'instance_id']], test_df[['user_id', 'real_first_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_first_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_first_category_str'] = temp_df['real_first_category'].map(lambda x: str(x))
    temp_df['user_real_first_category'] = temp_df['user_id_str'] + temp_df['real_first_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    historyShowList = []
    historyShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_first_category','date','show']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            historyShowList.append(np.sum(list(historyShowTemp.values())))
            historyShowTemp[dt] = show
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:show}
    tempDf['history_sameFirstCategory_count'] = historyShowList
    temp = pd.merge(temp_df[['user_real_first_category', 'date']], tempDf[['user_real_first_category', 'date', 'history_sameFirstCategory_count']], on = ['user_real_first_category', 'date'], how = 'left')
    train_df['history_sameFirstCategory_count'] = temp['history_sameFirstCategory_count'][:len(train_df['user_id'])]
    test_df['history_sameFirstCategory_count'] = temp['history_sameFirstCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getHistorySameFirstCategoryCount(train_df, test_df)
print(train_df[['date', 'history_sameFirstCategory_count', 'lastOneHour_sameFirstCategory_count']][train_df.lastOneHour_sameFirstCategory_count > 0].tail(10))


                      date  history_sameFirstCategory_count  \
478113 2018-09-24 15:40:13                                2   
478114 2018-09-24 01:33:20                                8   
478117 2018-09-24 09:07:38                                2   
478119 2018-09-24 22:47:56                                2   
478120 2018-09-24 22:44:11                                1   
478126 2018-09-24 14:20:07                                1   
478129 2018-09-24 14:25:29                                3   
478133 2018-09-24 07:48:39                                2   
478134 2018-09-24 23:03:35                                6   
478136 2018-09-24 18:08:49                                7   

        lastOneHour_sameFirstCategory_count  
478113                                  2.0  
478114                                  5.0  
478117                                  1.0  
478119                                  2.0  
478120                                  1.0  
478126                        

In [35]:
# 统计历史记录中某用户点击某个相同叶子类目商品的次数，同时对训练集和测试集进行处理
def getHistorySameLastCategoryCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_last_category', 'date', 'instance_id']], test_df[['user_id', 'real_last_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_last_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_last_category_str'] = temp_df['real_last_category'].map(lambda x: str(x))
    temp_df['user_real_last_category'] = temp_df['user_id_str'] + temp_df['real_last_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    historyShowList = []
    historyShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_last_category','date','show']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            historyShowList.append(np.sum(list(historyShowTemp.values())))
            historyShowTemp[dt] = show
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:show}
    tempDf['history_sameLastCategory_count'] = historyShowList
    temp = pd.merge(temp_df[['user_real_last_category', 'date']], tempDf[['user_real_last_category', 'date', 'history_sameLastCategory_count']], on = ['user_real_last_category', 'date'], how = 'left')
    train_df['history_sameLastCategory_count'] = temp['history_sameLastCategory_count'][:len(train_df['user_id'])]
    test_df['history_sameLastCategory_count'] = temp['history_sameLastCategory_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getHistorySameLastCategoryCount(train_df, test_df)
print(train_df[['date', 'history_sameLastCategory_count', 'lastOneHour_sameLastCategory_count']][train_df.lastOneHour_sameLastCategory_count > 0].tail(10))


                      date  history_sameLastCategory_count  \
478113 2018-09-24 15:40:13                               2   
478114 2018-09-24 01:33:20                               8   
478117 2018-09-24 09:07:38                               2   
478119 2018-09-24 22:47:56                               2   
478120 2018-09-24 22:44:11                               1   
478126 2018-09-24 14:20:07                               1   
478129 2018-09-24 14:25:29                               3   
478133 2018-09-24 07:48:39                               2   
478134 2018-09-24 23:03:35                               6   
478136 2018-09-24 18:08:49                               7   

        lastOneHour_sameLastCategory_count  
478113                                 2.0  
478114                                 5.0  
478117                                 1.0  
478119                                 2.0  
478120                                 1.0  
478126                                 1.0  
47

In [36]:
# 统计历史记录中某用户点击某个相同品牌商品的次数，同时对训练集和测试集进行处理
def getHistorySameBrandCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_brand_id', 'date', 'instance_id']], test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_brand_id_str'] = temp_df['item_brand_id'].map(lambda x: str(x))
    temp_df['user_item_brand_id'] = temp_df['user_id_str'] + temp_df['item_brand_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_brand_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_brand_id'] = tempDf['user_item_brand_id'].shift(1)
    tempDf['last_user_item_brand_id'] = tempDf['last_user_item_brand_id']==tempDf['user_item_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_brand_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    historyShowList = []
    historyShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_brand_id','date','show']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            historyShowList.append(np.sum(list(historyShowTemp.values())))
            historyShowTemp[dt] = show
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:show}
    tempDf['history_sameBrand_count'] = historyShowList
    temp = pd.merge(temp_df[['user_item_brand_id', 'date']], tempDf[['user_item_brand_id', 'date', 'history_sameBrand_count']], on = ['user_item_brand_id', 'date'], how = 'left')
    train_df['history_sameBrand_count'] = temp['history_sameBrand_count'][:len(train_df['user_id'])]
    test_df['history_sameBrand_count'] = temp['history_sameBrand_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getHistorySameBrandCount(train_df, test_df)
print(train_df[['date', 'history_sameBrand_count', 'lastOneHour_sameBrand_count']][train_df.lastOneHour_sameBrand_count > 0].tail(10))


                      date  history_sameBrand_count  \
478077 2018-09-24 16:23:00                        2   
478089 2018-09-24 09:01:35                        1   
478096 2018-09-24 15:41:39                        1   
478097 2018-09-24 17:18:19                        1   
478099 2018-09-24 23:23:22                        4   
478100 2018-09-24 22:11:40                        2   
478102 2018-09-24 21:55:55                        1   
478104 2018-09-24 22:23:31                        3   
478112 2018-09-24 16:46:49                        1   
478129 2018-09-24 14:25:29                        1   

        lastOneHour_sameBrand_count  
478077                          2.0  
478089                          1.0  
478096                          1.0  
478097                          1.0  
478099                          1.0  
478100                          2.0  
478102                          1.0  
478104                          2.0  
478112                          1.0  
478129        

In [37]:
# 统计历史记录中某用户点击某个相同店铺商品的次数，同时对训练集和测试集进行处理
def getHistorySameShopCount(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'shop_id', 'date', 'instance_id']], test_df[['user_id', 'shop_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'shop_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['shop_id_str'] = temp_df['shop_id'].map(lambda x: str(x))
    temp_df['user_shop_id'] = temp_df['user_id_str'] + temp_df['shop_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_shop_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_shop_id'] = tempDf['user_shop_id'].shift(1)
    tempDf['last_user_shop_id'] = tempDf['last_user_shop_id']==tempDf['user_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_shop_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    historyShowList = []
    historyShowTemp = {}
    for same, dt, show in tempDf[['last_user_shop_id','date','show']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            historyShowList.append(np.sum(list(historyShowTemp.values())))
            historyShowTemp[dt] = show
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:show}
    tempDf['history_sameShop_count'] = historyShowList
    temp = pd.merge(temp_df[['user_shop_id', 'date']], tempDf[['user_shop_id', 'date', 'history_sameShop_count']], on = ['user_shop_id', 'date'], how = 'left')
    train_df['history_sameShop_count'] = temp['history_sameShop_count'][:len(train_df['user_id'])]
    test_df['history_sameShop_count'] = temp['history_sameShop_count'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getHistorySameShopCount(train_df, test_df)
print(train_df[['date', 'history_sameShop_count', 'lastOneHour_sameShop_count']][train_df.lastOneHour_sameShop_count > 0].tail(10))


                      date  history_sameShop_count  lastOneHour_sameShop_count
477919 2018-09-24 02:17:00                       1                         1.0
477967 2018-09-24 17:04:13                       1                         1.0
477974 2018-09-24 21:43:14                       1                         1.0
477976 2018-09-24 10:09:56                       1                         1.0
478055 2018-09-24 21:37:25                       1                         1.0
478099 2018-09-24 23:23:22                       4                         1.0
478100 2018-09-24 22:11:40                       2                         2.0
478102 2018-09-24 21:55:55                       1                         1.0
478104 2018-09-24 22:23:31                       3                         2.0
478129 2018-09-24 14:25:29                       1                         1.0


In [38]:
# 统计历史记录中某用户是否购买过相同商品，同时对训练集和测试集进行处理
def getIsHistorySameItem(train_df, test_df):
    test_df_copy = test_df.copy()
    test_df_copy['is_trade'] = 0
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'is_trade', 'instance_id']], test_df_copy[['user_id', 'item_id', 'date', 'is_trade', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'is_trade', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item', 'date' ,'is_trade'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item'] = tempDf['user_item'].shift(1)
    tempDf['last_user_item'] = tempDf['last_user_item']==tempDf['user_item']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt, is_trade in tempDf[['last_user_item','date','is_trade']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
            else:
                historyShowList.append(0)
                if is_trade == 1:
                    historyShowTemp[dt] = is_trade
        else:
            historyShowList.append(0)
            if is_trade == 1:
                historyShowTemp = {dt:is_trade}
            else:
                historyShowTemp = {}
    tempDf['is_history_sameItem'] = historyShowList
    temp = pd.merge(temp_df[['user_item', 'date', 'is_trade']], tempDf[['user_item', 'date', 'is_trade', 'is_history_sameItem']], on = ['user_item', 'date' ,'is_trade'], how = 'left')
#     check_df = pd.pivot_table(temp_df, index=['user_item'], values=['instance_id'], aggfunc=len)
#     check_df.columns = ['show']
#     check_df.reset_index(inplace=True)
#     print(check_df[['user_item']][check_df.show > 10].head(10))
#     print(temp[['user_item', 'date', 'is_history_sameItem', 'is_trade']][temp.is_history_sameItem != 0].head(30))
#     print(temp[['user_item', 'date', 'is_history_sameItem', 'is_trade']][temp.user_item == '2414798119892047935410526681843914464'].head(10))
    train_df['is_history_sameItem'] = temp['is_history_sameItem'][:len(train_df['user_id'])]
    test_df['is_history_sameItem'] = temp['is_history_sameItem'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistorySameItem(train_df, test_df)
print(train_df[['date', 'is_history_sameItem', 'is_trade']].head(10))
print(len(train_df[['date', 'is_history_sameItem', 'is_trade']][train_df.is_history_sameItem == 1]))
print(len(train_df[['date', 'is_history_sameItem', 'is_trade']][((train_df.is_history_sameItem == 1) & (train_df.is_trade == 0))]))


                 date  is_history_sameItem  is_trade
0 2018-09-18 10:09:04                    0         0
1 2018-09-18 12:00:32                    0         0
2 2018-09-18 03:04:12                    0         0
3 2018-09-18 06:17:50                    0         0
4 2018-09-18 19:48:40                    0         0
5 2018-09-18 23:00:55                    0         0
6 2018-09-18 22:18:37                    0         0
7 2018-09-18 16:58:40                    0         0
8 2018-09-18 02:27:51                    0         0
9 2018-09-18 23:43:10                    0         0
188
171


In [39]:
# 统计历史记录中某用户是否购买过根类目的商品，同时对训练集和测试集进行处理
def getIsHistorySameFirstCategory(train_df, test_df):
    test_df_copy = test_df.copy()
    test_df_copy['is_trade'] = 0
    df = pd.concat([train_df[['user_id', 'real_first_category', 'date', 'is_trade', 'instance_id']], test_df_copy[['user_id', 'real_first_category', 'date', 'is_trade', 'instance_id']]])
    temp_df = df[['user_id', 'real_first_category', 'date', 'is_trade', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_first_category_str'] = temp_df['real_first_category'].map(lambda x: str(x))
    temp_df['user_real_first_category'] = temp_df['user_id_str'] + temp_df['real_first_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category', 'date' ,'is_trade'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt, is_trade in tempDf[['last_user_real_first_category','date','is_trade']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
            else:
                historyShowList.append(0)
                if is_trade == 1:
                    historyShowTemp[dt] = is_trade
        else:
            historyShowList.append(0)
            if is_trade == 1:
                historyShowTemp = {dt:is_trade}
            else:
                historyShowTemp = {}
    tempDf['is_history_sameFirstCategory'] = historyShowList
    temp = pd.merge(temp_df[['user_real_first_category', 'date', 'is_trade']], tempDf[['user_real_first_category', 'date', 'is_trade', 'is_history_sameFirstCategory']], on = ['user_real_first_category', 'date' ,'is_trade'], how = 'left')
#     check_df = pd.pivot_table(temp_df, index=['user_item'], values=['instance_id'], aggfunc=len)
#     check_df.columns = ['show']
#     check_df.reset_index(inplace=True)
#     print(check_df[['user_item']][check_df.show > 10].head(10))
#     print(temp[['user_item', 'date', 'is_history_sameItem', 'is_trade']][temp.is_history_sameItem != 0].head(30))
#     print(temp[['user_item', 'date', 'is_history_sameItem', 'is_trade']][temp.user_item == '2414798119892047935410526681843914464'].head(10))
    train_df['is_history_sameFirstCategory'] = temp['is_history_sameFirstCategory'][:len(train_df['user_id'])]
    test_df['is_history_sameFirstCategory'] = temp['is_history_sameFirstCategory'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistorySameFirstCategory(train_df, test_df)
print(train_df[['date', 'is_history_sameFirstCategory', 'is_trade']][train_df.is_history_sameFirstCategory == 1].head(10))
print(len(train_df[['date']][train_df.is_history_sameFirstCategory == 1]))
print(len(train_df[['date']][((train_df.is_history_sameFirstCategory == 1) & (train_df.is_trade == 0))]))


                    date  is_history_sameFirstCategory  is_trade
303  2018-09-18 21:01:22                             1         0
511  2018-09-18 13:51:00                             1         0
556  2018-09-18 23:43:22                             1         0
612  2018-09-18 23:37:40                             1         0
762  2018-09-18 20:36:35                             1         0
924  2018-09-18 21:19:48                             1         0
1003 2018-09-18 18:40:38                             1         0
1012 2018-09-18 12:55:38                             1         0
1298 2018-09-18 14:36:22                             1         0
1627 2018-09-18 21:18:30                             1         0
6001
5798


In [40]:
# 统计历史记录中某用户是否购买过叶子类目的商品，同时对训练集和测试集进行处理
def getIsHistorySameLastCategory(train_df, test_df):
    test_df_copy = test_df.copy()
    test_df_copy['is_trade'] = 0
    df = pd.concat([train_df[['user_id', 'real_last_category', 'date', 'is_trade', 'instance_id']], test_df_copy[['user_id', 'real_last_category', 'date', 'is_trade', 'instance_id']]])
    temp_df = df[['user_id', 'real_last_category', 'date', 'is_trade', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_last_category_str'] = temp_df['real_last_category'].map(lambda x: str(x))
    temp_df['user_real_last_category'] = temp_df['user_id_str'] + temp_df['real_last_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date' ,'is_trade'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt, is_trade in tempDf[['last_user_real_last_category','date','is_trade']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
            else:
                historyShowList.append(0)
                if is_trade == 1:
                    historyShowTemp[dt] = is_trade
        else:
            historyShowList.append(0)
            if is_trade == 1:
                historyShowTemp = {dt:is_trade}
            else:
                historyShowTemp = {}
    tempDf['is_history_sameLastCategory'] = historyShowList
    temp = pd.merge(temp_df[['user_real_last_category', 'date', 'is_trade']], tempDf[['user_real_last_category', 'date', 'is_trade', 'is_history_sameLastCategory']], on = ['user_real_last_category', 'date' ,'is_trade'], how = 'left')
    train_df['is_history_sameLastCategory'] = temp['is_history_sameLastCategory'][:len(train_df['user_id'])]
    test_df['is_history_sameLastCategory'] = temp['is_history_sameLastCategory'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistorySameLastCategory(train_df, test_df)
print(train_df[['date', 'is_history_sameLastCategory', 'is_trade']][train_df.is_history_sameLastCategory == 1].head(10))
print(len(train_df[['date']][train_df.is_history_sameLastCategory == 1]))
print(len(train_df[['date']][((train_df.is_history_sameLastCategory == 1) & (train_df.is_trade == 0))]))


                    date  is_history_sameLastCategory  is_trade
303  2018-09-18 21:01:22                            1         0
511  2018-09-18 13:51:00                            1         0
556  2018-09-18 23:43:22                            1         0
612  2018-09-18 23:37:40                            1         0
762  2018-09-18 20:36:35                            1         0
924  2018-09-18 21:19:48                            1         0
1003 2018-09-18 18:40:38                            1         0
1012 2018-09-18 12:55:38                            1         0
1298 2018-09-18 14:36:22                            1         0
1627 2018-09-18 21:18:30                            1         0
5955
5754


In [41]:
# 统计历史记录中某用户是否购买过相同品牌的商品，同时对训练集和测试集进行处理
def getIsHistorySameBrand(train_df, test_df):
    test_df_copy = test_df.copy()
    test_df_copy['is_trade'] = 0
    df = pd.concat([train_df[['user_id', 'item_brand_id', 'date', 'is_trade', 'instance_id']], test_df_copy[['user_id', 'item_brand_id', 'date', 'is_trade', 'instance_id']]])
    temp_df = df[['user_id', 'item_brand_id', 'date', 'is_trade', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_brand_id_str'] = temp_df['item_brand_id'].map(lambda x: str(x))
    temp_df['user_item_brand_id'] = temp_df['user_id_str'] + temp_df['item_brand_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_brand_id', 'date' ,'is_trade'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_brand_id'] = tempDf['user_item_brand_id'].shift(1)
    tempDf['last_user_item_brand_id'] = tempDf['last_user_item_brand_id']==tempDf['user_item_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_brand_id, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt, is_trade in tempDf[['last_user_item_brand_id','date','is_trade']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
            else:
                historyShowList.append(0)
                if is_trade == 1:
                    historyShowTemp[dt] = is_trade
        else:
            historyShowList.append(0)
            if is_trade == 1:
                historyShowTemp = {dt:is_trade}
            else:
                historyShowTemp = {}
    tempDf['is_history_sameBrand'] = historyShowList
    temp = pd.merge(temp_df[['user_item_brand_id', 'date', 'is_trade']], tempDf[['user_item_brand_id', 'date', 'is_trade', 'is_history_sameBrand']], on = ['user_item_brand_id', 'date' ,'is_trade'], how = 'left')
    train_df['is_history_sameBrand'] = temp['is_history_sameBrand'][:len(train_df['user_id'])]
    test_df['is_history_sameBrand'] = temp['is_history_sameBrand'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistorySameBrand(train_df, test_df)
print(train_df[['date', 'is_history_sameBrand', 'is_trade']][train_df.is_history_sameBrand == 1].head(10))
print(len(train_df[['date']][train_df.is_history_sameBrand == 1]))
print(len(train_df[['date']][((train_df.is_history_sameBrand == 1) & (train_df.is_trade == 0))]))


                    date  is_history_sameBrand  is_trade
303  2018-09-18 21:01:22                     1         0
1012 2018-09-18 12:55:38                     1         0
1298 2018-09-18 14:36:22                     1         0
2486 2018-09-18 12:08:58                     1         0
2512 2018-09-18 10:03:50                     1         0
2515 2018-09-18 08:14:21                     1         1
2596 2018-09-18 10:39:04                     1         0
2813 2018-09-18 08:01:12                     1         0
3193 2018-09-18 12:08:03                     1         0
3309 2018-09-18 12:06:21                     1         1
608
574


In [42]:
# 统计历史记录中某用户是否购买过相同店铺的商品，同时对训练集和测试集进行处理
def getIsHistorySameShop(train_df, test_df):
    test_df_copy = test_df.copy()
    test_df_copy['is_trade'] = 0
    df = pd.concat([train_df[['user_id', 'shop_id', 'date', 'is_trade', 'instance_id']], test_df_copy[['user_id', 'shop_id', 'date', 'is_trade', 'instance_id']]])
    temp_df = df[['user_id', 'shop_id', 'date', 'is_trade', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['shop_id_str'] = temp_df['shop_id'].map(lambda x: str(x))
    temp_df['user_shop_id'] = temp_df['user_id_str'] + temp_df['shop_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_shop_id', 'date' ,'is_trade'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_shop_id'] = tempDf['user_shop_id'].shift(1)
    tempDf['last_user_shop_id'] = tempDf['last_user_shop_id']==tempDf['user_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_shop_id, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt, is_trade in tempDf[['last_user_shop_id','date','is_trade']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
            else:
                historyShowList.append(0)
                if is_trade == 1:
                    historyShowTemp[dt] = is_trade
        else:
            historyShowList.append(0)
            if is_trade == 1:
                historyShowTemp = {dt:is_trade}
            else:
                historyShowTemp = {}
    tempDf['is_history_sameShop'] = historyShowList
    temp = pd.merge(temp_df[['user_shop_id', 'date', 'is_trade']], tempDf[['user_shop_id', 'date', 'is_trade', 'is_history_sameShop']], on = ['user_shop_id', 'date' ,'is_trade'], how = 'left')
    train_df['is_history_sameShop'] = temp['is_history_sameShop'][:len(train_df['user_id'])]
    test_df['is_history_sameShop'] = temp['is_history_sameShop'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistorySameShop(train_df, test_df)
print(train_df[['date', 'is_history_sameShop', 'is_trade']][train_df.is_history_sameShop == 1].head(10))
print(len(train_df[['date']][train_df.is_history_sameShop == 1]))
print(len(train_df[['date']][((train_df.is_history_sameShop == 1) & (train_df.is_trade == 0))]))


                    date  is_history_sameShop  is_trade
303  2018-09-18 21:01:22                    1         0
1298 2018-09-18 14:36:22                    1         0
2512 2018-09-18 10:03:50                    1         0
2515 2018-09-18 08:14:21                    1         1
2596 2018-09-18 10:39:04                    1         0
2813 2018-09-18 08:01:12                    1         0
3309 2018-09-18 12:06:21                    1         1
3916 2018-09-18 20:52:47                    1         0
4655 2018-09-18 13:10:51                    1         0
4923 2018-09-18 21:56:11                    1         0
308
286


In [43]:
# 统计历史记录中某用户是否第一次点击该商品，同时对训练集和测试集进行处理
def getIsHistoryFirstClickItem(train_df, test_df):
    test_df_copy = test_df.copy()
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df_copy[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item_id'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_item_id','date']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k > dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(0)
            else:
                historyShowList.append(1)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(1)
            historyShowTemp = {dt:same}
    tempDf['is_history_firstClickItem'] = historyShowList
    temp = pd.merge(temp_df[['user_item_id', 'date']], tempDf[['user_item_id', 'date', 'is_history_firstClickItem']], on = ['user_item_id', 'date'], how = 'left')
    train_df['is_history_firstClickItem'] = temp['is_history_firstClickItem'][:len(train_df['user_id'])]
    test_df['is_history_firstClickItem'] = temp['is_history_firstClickItem'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getIsHistoryFirstClickItem(train_df, test_df)
print(train_df[['date', 'is_history_firstClickItem', 'is_trade']][train_df.is_history_firstClickItem == 1].head(10))
print(len(train_df[((train_df.is_history_firstClickItem == 1) & (train_df.is_trade == 1))]))
print(len(train_df[((train_df.is_history_firstClickItem == 1) & (train_df.is_trade == 0))]))
print(len(train_df[((train_df.is_history_firstClickItem == 0) & (train_df.is_trade == 1))]))
print(len(train_df[((train_df.is_history_firstClickItem == 0) & (train_df.is_trade == 0))]))


                 date  is_history_firstClickItem  is_trade
0 2018-09-18 10:09:04                          1         0
1 2018-09-18 12:00:32                          1         0
2 2018-09-18 03:04:12                          1         0
3 2018-09-18 06:17:50                          1         0
4 2018-09-18 19:48:40                          1         0
5 2018-09-18 23:00:55                          1         0
6 2018-09-18 22:18:37                          1         0
7 2018-09-18 16:58:40                          1         0
8 2018-09-18 02:27:51                          1         0
9 2018-09-18 23:43:10                          1         0
7698
436581
1323
32536


In [44]:
# 统计历史记录中某用户后面是否有点击相同商品，同时对训练集和测试集进行处理
def getIsClickSameItemLater(train_df, test_df):
    train_df_copy = train_df[['user_id', 'item_id', 'date', 'instance_id']]
    print(len(train_df_copy))
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    print(len(test_df_copy))
    train_df_copy['user_id_str'] = train_df_copy['user_id'].map(lambda x: str(x))
    train_df_copy['item_id_str'] = train_df_copy['item_id'].map(lambda x: str(x))
    train_df_copy['user_item_id'] = train_df_copy['user_id_str'] + train_df_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_copy, test_df_copy])
#     temp_df = temp_df.sort_values(by='date', ascending=False)
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    print(tempDf.info())
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_item_id', 'date'], ascending=False)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
#     print(tempDf.head(30))
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_item_id','date']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k < dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['is_later_clickSameItem'] = historyShowList
#     temp = pd.merge(temp_df[['user_item_id', 'date', 'user_id', 'item_id']], tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how = 'left')
    train_df_copy = train_df_copy.merge(tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], how='left', on=['user_item_id', 'date'])
    print(len(train_df_copy))
    train_df['is_later_clickSameItem'] = train_df_copy['is_later_clickSameItem']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how='left')
    print(len(test_df_copy))
    test_df['is_later_clickSameItem'] = test_df_copy['is_later_clickSameItem']
    #     print(train_df[['date', 'is_later_clickSameItem', 'user_item_id']][train_df.user_item_id == '9996063195993229603233038791884113820'].head(5))
    return train_df, test_df

train_df, test_df = getIsClickSameItemLater(train_df, test_df)



478138
61259


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 539370 entries, (10000468422575080321207563772024247788, 2018-09-20 09:44:00) to (9999524220176225161121649958974113109, 2018-09-23 21:35:50)
Data columns (total 1 columns):
instance_id    539370 non-null int64
dtypes: int64(1)
memory usage: 14.5+ MB
None
478138
61259


In [45]:
# 统计历史记录中某用户后面是否有点击相同叶子类目商品，同时对训练集和测试集进行处理
def getIsClickSameLastCategoryLater(train_df, test_df):
    train_df_copy = train_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    print(len(train_df_copy))
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    print(len(test_df_copy))
    train_df_copy['user_id_str'] = train_df_copy['user_id'].map(lambda x: str(x))
    train_df_copy['real_last_category_str'] = train_df_copy['real_last_category'].map(lambda x: str(x))
    train_df_copy['user_real_last_category'] = train_df_copy['user_id_str'] + train_df_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_copy, test_df_copy])
#     temp_df = temp_df.sort_values(by='date', ascending=False)
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    print(tempDf.info())
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_real_last_category', 'date'], ascending=False)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
#     print(tempDf.head(30))
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_real_last_category','date']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k < dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(1)
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['is_later_clickSameLastCategory'] = historyShowList
#     temp = pd.merge(temp_df[['user_item_id', 'date', 'user_id', 'item_id']], tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how = 'left')
    train_df_copy = train_df_copy.merge(tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], how='left', on=['user_real_last_category', 'date'])
    print(len(train_df_copy))
    train_df['is_later_clickSameLastCategory'] = train_df_copy['is_later_clickSameLastCategory']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], on = ['user_real_last_category', 'date'], how='left')
    print(len(test_df_copy))
    test_df['is_later_clickSameLastCategory'] = test_df_copy['is_later_clickSameLastCategory']
    return train_df, test_df
    

train_df, test_df = getIsClickSameLastCategoryLater(train_df, test_df)



478138
61259


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 498674 entries, (10000468422575080329, 2018-09-20 09:44:00) to (9999524220176225169, 2018-09-23 21:35:50)
Data columns (total 1 columns):
instance_id    498674 non-null int64
dtypes: int64(1)
memory usage: 11.7+ MB
None
478138
61259


In [46]:
# 统计历史记录中某用户后面点击相同叶子类目商品的个数，同时对训练集和测试集进行处理
def getClickSameLastCategoryLaterNumber(train_df, test_df):
    train_df_copy = train_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    print(len(train_df_copy))
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    print(len(test_df_copy))
    train_df_copy['user_id_str'] = train_df_copy['user_id'].map(lambda x: str(x))
    train_df_copy['real_last_category_str'] = train_df_copy['real_last_category'].map(lambda x: str(x))
    train_df_copy['user_real_last_category'] = train_df_copy['user_id_str'] + train_df_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_copy, test_df_copy])
#     temp_df = temp_df.sort_values(by='date', ascending=False)
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    print(tempDf.info())
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_real_last_category', 'date'], ascending=False)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
#     print(tempDf.head(30))
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_real_last_category','date']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k < dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(len(historyShowTemp))
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['later_clickSameLastCategory_count'] = historyShowList
#     temp = pd.merge(temp_df[['user_item_id', 'date', 'user_id', 'item_id']], tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how = 'left')
    train_df_copy = train_df_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    print(len(train_df_copy))
    train_df['later_clickSameLastCategory_count'] = train_df_copy['later_clickSameLastCategory_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], on = ['user_real_last_category', 'date'], how='left')
    print(len(test_df_copy))
    test_df['later_clickSameLastCategory_count'] = test_df_copy['later_clickSameLastCategory_count']
    return train_df, test_df

train_df, test_df = getClickSameLastCategoryLaterNumber(train_df, test_df)



478138
61259


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 498674 entries, (10000468422575080329, 2018-09-20 09:44:00) to (9999524220176225169, 2018-09-23 21:35:50)
Data columns (total 1 columns):
instance_id    498674 non-null int64
dtypes: int64(1)
memory usage: 11.7+ MB
None
478138
61259


In [47]:
# 统计历史记录中某用户后面点击相同商品的个数，同时对训练集和测试集进行处理
def getClickSameItemLaterNumber(train_df, test_df):
    train_df_copy = train_df[['user_id', 'item_id', 'date', 'instance_id']]
    print(len(train_df_copy))
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    print(len(test_df_copy))
    train_df_copy['user_id_str'] = train_df_copy['user_id'].map(lambda x: str(x))
    train_df_copy['item_id_str'] = train_df_copy['item_id'].map(lambda x: str(x))
    train_df_copy['user_item_id'] = train_df_copy['user_id_str'] + train_df_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_copy, test_df_copy])
#     temp_df = temp_df.sort_values(by='date', ascending=False)
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    print(tempDf.info())
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_item_id', 'date'], ascending=False)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
#     print(tempDf.head(30))
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_item_id','date']].values:
        if same:
            [historyShowTemp.pop(k) for k in list(historyShowTemp) if k < dt]
            if len(historyShowTemp) > 0:
                historyShowList.append(len(historyShowTemp))
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['later_clickSameItem_count'] = historyShowList
#     temp = pd.merge(temp_df[['user_item_id', 'date', 'user_id', 'item_id']], tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how = 'left')
    train_df_copy = train_df_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], how='left', on=['user_item_id', 'date'])
    print(len(train_df_copy))
    train_df['later_clickSameItem_count'] = train_df_copy['later_clickSameItem_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], on = ['user_item_id', 'date'], how='left')
    print(len(test_df_copy))
    test_df['later_clickSameItem_count'] = test_df_copy['later_clickSameItem_count']
    #     print(train_df[['date', 'is_later_clickSameItem', 'user_item_id']][train_df.user_item_id == '9996063195993229603233038791884113820'].head(5))
    return train_df, test_df
    

train_df, test_df = getClickSameItemLaterNumber(train_df, test_df)



478138
61259


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 539370 entries, (10000468422575080321207563772024247788, 2018-09-20 09:44:00) to (9999524220176225161121649958974113109, 2018-09-23 21:35:50)
Data columns (total 1 columns):
instance_id    539370 non-null int64
dtypes: int64(1)
memory usage: 14.5+ MB
None
478138
61259


In [48]:
#构造跟店铺信息相关的特征，评分是否高于平均水平，平均水平根据是否是同一个第一类目来判定
def getIsHighFeture(train_df, test_df, colName):
    
    train_df[colName] = train_df[colName].map(lambda x: -1 if math.isnan(x) else x)
    train_df['real_first_category'] = train_df.item_category_list.map(lambda x: x.split(';')[1])
    notmiss_train_df = train_df[train_df[colName] != -1]
    train_df_pivot_table = pd.pivot_table(notmiss_train_df[['real_first_category', 'shop_review_positive_rate']], index=['real_first_category'], values=['shop_review_positive_rate'], aggfunc=np.mean)
    train_df_pivot_table.columns = [colName + '_ave']
    train_df_pivot_table.reset_index(inplace=True)

    notmiss_train_df = pd.merge(notmiss_train_df, train_df_pivot_table, on=['real_first_category'], how='left')
    notmiss_train_df[colName + '_diff'] = notmiss_train_df[colName] - notmiss_train_df[colName + '_ave']
    notmiss_train_df['isHigh_' + colName] = notmiss_train_df[colName + '_diff'].map(lambda x: 1 if x >= 0 else 0)

    train_df['isHigh_' + colName] = 2
    train_df['isHigh_' + colName][train_df[colName] != -1] = notmiss_train_df['isHigh_' + colName]
    train_df[colName + '_ave'] = -1
    train_df[colName + '_ave'][train_df[colName] != -1] = notmiss_train_df[colName + '_ave']
    train_df[colName + '_diff'] = -1
    train_df[colName + '_diff'][train_df[colName] != -1] = notmiss_train_df[colName + '_diff']

    test_df[colName] = test_df[colName].map(lambda x: -1 if math.isnan(x) else x)
    test_df['real_first_category'] = test_df.item_category_list.map(lambda x: x.split(';')[1])
    notmiss_test_df = test_df[test_df[colName] != -1]
    notmiss_test_df = pd.merge(notmiss_test_df, train_df_pivot_table, on=['real_first_category'], how='left')
    notmiss_test_df[colName + '_ave'] = notmiss_test_df[colName + '_ave'].map(lambda x: -1 if x == np.nan else x)
    test_df[colName + '_ave'] = -1
    test_df[colName + '_ave'][test_df[colName] != -1] = notmiss_test_df[colName + '_ave']
    test_df[colName + '_diff'] = -1
    test_df[colName + '_diff'][test_df[colName + '_ave'] != -1] = test_df[colName][test_df[colName + '_ave'] != -1] - test_df[colName + '_ave'][test_df[colName + '_ave'] != -1]
    test_df['isHigh_' + colName] = 2
    test_df['isHigh_' + colName][test_df[colName + '_diff'] >= 0] = 1
    test_df['isHigh_' + colName][test_df[colName + '_diff'] < 0] = 0
    return train_df, test_df

#添加特征
train_df, test_df = getIsHighFeture(train_df, test_df, 'shop_review_positive_rate')
train_df, test_df = getIsHighFeture(train_df, test_df, 'shop_score_service')
train_df, test_df = getIsHighFeture(train_df, test_df, 'shop_score_delivery')
train_df, test_df = getIsHighFeture(train_df, test_df, 'shop_score_description')

print(train_df[['shop_review_positive_rate', 'shop_review_positive_rate_ave', 'isHigh_shop_review_positive_rate']].head(10))
print(test_df[['real_first_category', 'shop_review_positive_rate', 'shop_review_positive_rate_ave', 'isHigh_shop_review_positive_rate']].head(10))


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A val

   shop_review_positive_rate  shop_review_positive_rate_ave  \
0                   1.000000                       0.991042   
1                   1.000000                       0.991042   
2                   1.000000                       0.991042   
3                   1.000000                       0.991042   
4                   1.000000                       0.991042   
5                   1.000000                       0.991042   
6                   0.985427                       0.995915   
7                   0.985427                       0.995915   
8                   0.985427                       0.996590   
9                   0.985427                       0.995915   

   isHigh_shop_review_positive_rate  
0                               1.0  
1                               1.0  
2                               1.0  
3                               1.0  
4                               1.0  
5                               1.0  
6                               0.0  
7 

In [49]:
# 统计某用户距离上次点击相同商品的时间，同时对训练集和测试集进行处理
def getUserItemLastClickDeltaTime(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_id', 'date', 'instance_id']], test_df[['user_id', 'item_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item_id'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_item_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userItem_lastClickDeltaTime'] = historyShowList
#     print(tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']].head(10))
    temp = pd.merge(temp_df[['user_item_id', 'date']], tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']], on = ['user_item_id', 'date'], how = 'left')
    train_df['userItem_lastClickDeltaTime'] = temp['userItem_lastClickDeltaTime'][:len(train_df['user_id'])]
    test_df['userItem_lastClickDeltaTime'] = temp['userItem_lastClickDeltaTime'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getUserItemLastClickDeltaTime(train_df, test_df)
print(train_df[['user_id', 'item_id', 'date', 'userItem_lastClickDeltaTime']].head(10))


               user_id              item_id                date  \
0  4505772604969228686  3412720377098676069 2018-09-18 10:09:04   
1  2692638157208937547  3412720377098676069 2018-09-18 12:00:32   
2  5247924392014515924  3412720377098676069 2018-09-18 03:04:12   
3  2681414445369714628  3412720377098676069 2018-09-18 06:17:50   
4  2729475788342039013  3412720377098676069 2018-09-18 19:48:40   
5  4512655448325954611  3412720377098676069 2018-09-18 23:00:55   
6  8811056487516803043   285660928590172217 2018-09-18 22:18:37   
7  6507704883896466138   285660928590172217 2018-09-18 16:58:40   
8  6203308008480593423  5202355029344881809 2018-09-18 02:27:51   
9  6041712044514783312   285660928590172217 2018-09-18 23:43:10   

   userItem_lastClickDeltaTime  
0                   99999999.0  
1                   99999999.0  
2                   99999999.0  
3                   99999999.0  
4                   99999999.0  
5                   99999999.0  
6                   99999999.0 

In [50]:
# 统计某用户距离上次点击相同品牌商品的时间，同时对训练集和测试集进行处理
def getUserBrandLastClickDeltaTime(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'item_brand_id', 'date', 'instance_id']], test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_brand_id_str'] = temp_df['item_brand_id'].map(lambda x: str(x))
    temp_df['user_item_brand_id'] = temp_df['user_id_str'] + temp_df['item_brand_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_brand_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_brand_id'] = tempDf['user_item_brand_id'].shift(1)
    tempDf['last_user_item_brand_id'] = tempDf['last_user_item_brand_id']==tempDf['user_item_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_brand_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_item_brand_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userBrand_lastClickDeltaTime'] = historyShowList
    print(tempDf[['user_item_brand_id', 'date', 'userBrand_lastClickDeltaTime']].head(10))
    temp = pd.merge(temp_df[['user_item_brand_id', 'date']], tempDf[['user_item_brand_id', 'date', 'userBrand_lastClickDeltaTime']], on = ['user_item_brand_id', 'date'], how = 'left')
    train_df['userBrand_lastClickDeltaTime'] = temp['userBrand_lastClickDeltaTime'][:len(train_df['user_id'])]
    test_df['userBrand_lastClickDeltaTime'] = temp['userBrand_lastClickDeltaTime'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getUserBrandLastClickDeltaTime(train_df, test_df)
print(train_df[['user_id', 'item_brand_id', 'date', 'userBrand_lastClickDeltaTime']].head(10))


                       user_item_brand_id                date  \
0  10000468422575080322479301133483350869 2018-09-20 09:44:00   
1  10000531910868977561850538188661123614 2018-09-22 17:53:17   
2  10000531910868977561869485793196349073 2018-09-22 09:52:49   
3  10000531910868977561869485793196349073 2018-09-22 17:49:03   
4  10000773166726440687838285046767229711 2018-09-23 16:44:11   
5  10001070043842559681114275315610038623 2018-09-25 21:58:26   
6  10001070043842559686400031259989360861 2018-09-25 21:56:32   
7  10001437354507214243953456177407100048 2018-09-23 09:37:24   
8  10001513848033675592348486371677245014 2018-09-22 21:49:53   
9  10001513848033675594191021222338422219 2018-09-22 21:46:59   

   userBrand_lastClickDeltaTime  
0                    99999999.0  
1                    99999999.0  
2                    99999999.0  
3                       28574.0  
4                    99999999.0  
5                    99999999.0  
6                    99999999.0  
7           

In [51]:
# 统计某用户距离上次点击相同店铺的时间，同时对训练集和测试集进行处理
def getUserShopLastClickDeltaTime(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'shop_id', 'date', 'instance_id']], test_df[['user_id', 'shop_id', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'shop_id', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['shop_id_str'] = temp_df['shop_id'].map(lambda x: str(x))
    temp_df['user_shop_id'] = temp_df['user_id_str'] + temp_df['shop_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_shop_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_shop_id'] = tempDf['user_shop_id'].shift(1)
    tempDf['last_user_shop_id'] = tempDf['last_user_shop_id']==tempDf['user_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_shop_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_shop_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userShop_lastClickDeltaTime'] = historyShowList
    print(tempDf[['user_shop_id', 'date', 'userShop_lastClickDeltaTime']].head(10))
    temp = pd.merge(temp_df[['user_shop_id', 'date']], tempDf[['user_shop_id', 'date', 'userShop_lastClickDeltaTime']], on = ['user_shop_id', 'date'], how = 'left')
    train_df['userShop_lastClickDeltaTime'] = temp['userShop_lastClickDeltaTime'][:len(train_df['user_id'])]
    test_df['userShop_lastClickDeltaTime'] = temp['userShop_lastClickDeltaTime'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getUserShopLastClickDeltaTime(train_df, test_df)
print(train_df[['user_id', 'shop_id', 'date', 'userShop_lastClickDeltaTime']].head(10))


                             user_shop_id                date  \
0  10000468422575080324571715481267183187 2018-09-20 09:44:00   
1  10000531910868977563526941526328534645 2018-09-22 09:52:49   
2  10000531910868977563526941526328534645 2018-09-22 17:49:03   
3  10000531910868977567681669516894696027 2018-09-22 17:53:17   
4  10000773166726440685239284608305401977 2018-09-23 16:44:11   
5  10001070043842559681091499661419284880 2018-09-25 21:58:26   
6  10001070043842559688884404577107716547 2018-09-25 21:56:32   
7  10001437354507214244142178404332280874 2018-09-23 09:37:24   
8   1000151384803367559500378403670609059 2018-09-22 21:50:40   
9  10001513848033675596635220515139352173 2018-09-22 21:46:59   

   userShop_lastClickDeltaTime  
0                   99999999.0  
1                   99999999.0  
2                      28574.0  
3                   99999999.0  
4                   99999999.0  
5                   99999999.0  
6                   99999999.0  
7                   

In [52]:
# 统计某用户距离上次点击相同根类目的时间，同时对训练集和测试集进行处理
def getUserFirstCategoryLastClickDeltaTime(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_first_category', 'date', 'instance_id']], test_df[['user_id', 'real_first_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_first_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_first_category_str'] = temp_df['real_first_category'].map(lambda x: str(x))
    temp_df['user_real_first_category'] = temp_df['user_id_str'] + temp_df['real_first_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_real_first_category', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userFirstCategory_lastClickDeltaTime'] = historyShowList
    print(tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']].head(10))
    temp = pd.merge(temp_df[['user_real_first_category', 'date']], tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']], on = ['user_real_first_category', 'date'], how = 'left')
    train_df['userFirstCategory_lastClickDeltaTime'] = temp['userFirstCategory_lastClickDeltaTime'][:len(train_df['user_id'])]
    test_df['userFirstCategory_lastClickDeltaTime'] = temp['userFirstCategory_lastClickDeltaTime'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getUserFirstCategoryLastClickDeltaTime(train_df, test_df)
print(train_df[['user_id', 'real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']].head(10))


                 user_real_first_category                date  \
0  10000468422575080325755694407684602296 2018-09-20 09:44:00   
1  10000531910868977565755694407684602296 2018-09-22 09:52:49   
2  10000531910868977565755694407684602296 2018-09-22 17:49:03   
3  10000531910868977565755694407684602296 2018-09-22 17:53:17   
4  10000773166726440688277336076276184272 2018-09-23 16:44:11   
5  10001070043842559688277336076276184272 2018-09-25 21:56:32   
6  10001070043842559688277336076276184272 2018-09-25 21:58:26   
7  10001437354507214248277336076276184272 2018-09-23 09:37:24   
8   1000151384803367559509660095530134768 2018-09-22 21:46:59   
9   1000151384803367559509660095530134768 2018-09-22 21:49:53   

   userFirstCategory_lastClickDeltaTime  
0                            99999999.0  
1                            99999999.0  
2                               28574.0  
3                                 254.0  
4                            99999999.0  
5                            999

In [53]:
# 统计某用户距离上次点击相同叶子类目的时间，同时对训练集和测试集进行处理
def getUserLastCategoryLastClickDeltaTime(train_df, test_df):
    df = pd.concat([train_df[['user_id', 'real_last_category', 'date', 'instance_id']], test_df[['user_id', 'real_last_category', 'date', 'instance_id']]])
    temp_df = df[['user_id', 'real_last_category', 'date', 'instance_id']]
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['real_last_category_str'] = temp_df['real_last_category'].map(lambda x: str(x))
    temp_df['user_real_last_category'] = temp_df['user_id_str'] + temp_df['real_last_category_str']
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_real_last_category', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userLastCategory_lastClickDeltaTime'] = historyShowList
    print(tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']].head(10))
    temp = pd.merge(temp_df[['user_real_last_category', 'date']], tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']], on = ['user_real_last_category', 'date'], how = 'left')
    train_df['userLastCategory_lastClickDeltaTime'] = temp['userLastCategory_lastClickDeltaTime'][:len(train_df['user_id'])]
    test_df['userLastCategory_lastClickDeltaTime'] = temp['userLastCategory_lastClickDeltaTime'][len(train_df['user_id']):]
    return train_df, test_df

train_df, test_df = getUserLastCategoryLastClickDeltaTime(train_df, test_df)
print(train_df[['user_id', 'real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']].head(10))


  user_real_last_category                date  \
0    10000468422575080329 2018-09-20 09:44:00   
1    10000531910868977569 2018-09-22 09:52:49   
2    10000531910868977569 2018-09-22 17:49:03   
3    10000531910868977569 2018-09-22 17:53:17   
4    10000773166726440689 2018-09-23 16:44:11   
5    10001070043842559689 2018-09-25 21:56:32   
6    10001070043842559689 2018-09-25 21:58:26   
7    10001437354507214249 2018-09-23 09:37:24   
8    10001513848033675599 2018-09-22 21:46:59   
9    10001513848033675599 2018-09-22 21:49:53   

   userLastCategory_lastClickDeltaTime  
0                           99999999.0  
1                           99999999.0  
2                              28574.0  
3                                254.0  
4                           99999999.0  
5                           99999999.0  
6                                114.0  
7                           99999999.0  
8                           99999999.0  
9                                174.0  
         

In [54]:
# 定义获取某种店铺，品牌，城市对应商品种类，用户数的函数
def getCorrespondNumber(train_df, test_df, colName1, colName2, newColName):
    df = pd.concat([train_df[[colName1, colName2, 'instance_id']], test_df[[colName1, colName2, 'instance_id']]])
    temp_df = df[[colName1, colName2, 'instance_id']]
    tempDf = temp_df.sort_values(by=colName1, ascending=False)
    tempDf['last_' + colName1] = tempDf[colName1].shift(1)
    tempDf['same'] = tempDf['last_' + colName1]==tempDf[colName1]
    
    colName1List = []
    countList = []
    colName2Set = set()
    for same, col2, last_col1 in tempDf[['same', colName2, 'last_' + colName1]].values:
        if same:
            colName2Set.add(col2)
        else:
            colName1List.append(last_col1)
            countList.append(len(colName2Set))
            colName2Set = {col2}
    #处理最后一行数据
    last_col1 = tempDf.iloc[-1][colName1]
    last_count = len(colName2Set)
    colName1List.append(last_col1)
    countList.append(last_count)
    #将结果组合到tempDf中
    result_df = {colName1: colName1List, newColName: countList}
    result_df = DataFrame(result_df)
    result_df = result_df[1:]
    tempDf = pd.merge(tempDf[[colName1, colName2]], result_df[[colName1, newColName]], on=[colName1])
#     print(tempDf[[colName1, colName2, newColName]].head(10))
    temp = pd.merge(temp_df[[colName1, colName2]], tempDf[[colName1, colName2, newColName]], on = [colName1, colName2])
    
    train_df[newColName] = temp[newColName][:len(train_df[colName1])]
    test_df[newColName] = temp[newColName][len(train_df[colName1]):]
    return train_df, test_df

train_df, test_df = getCorrespondNumber(train_df, test_df, 'shop_id', 'item_id', 'shop_item_classNumber')
train_df, test_df = getCorrespondNumber(train_df, test_df, 'item_brand_id', 'item_id', 'brand_item_classNumber')
train_df, test_df = getCorrespondNumber(train_df, test_df, 'item_city_id', 'item_id', 'city_item_classNumber')
train_df, test_df = getCorrespondNumber(train_df, test_df, 'shop_id', 'user_id', 'shop_user_classNumber')
train_df, test_df = getCorrespondNumber(train_df, test_df, 'item_brand_id', 'user_id', 'brand_user_classNumber')
train_df, test_df = getCorrespondNumber(train_df, test_df, 'item_city_id', 'user_id', 'city_user_classNumber')
print(train_df[['user_id', 'item_id', 'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber',
               'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber']].head(10))


               user_id              item_id  shop_item_classNumber  \
0  4505772604969228686  3412720377098676069                    3.0   
1  2692638157208937547  3412720377098676069                    3.0   
2  5247924392014515924  3412720377098676069                    3.0   
3  2681414445369714628  3412720377098676069                    3.0   
4  2729475788342039013  3412720377098676069                    3.0   
5  4512655448325954611  3412720377098676069                    3.0   
6  8811056487516803043   285660928590172217                    3.0   
7  6507704883896466138   285660928590172217                    3.0   
8  6203308008480593423  5202355029344881809                    3.0   
9  6041712044514783312   285660928590172217                    3.0   

   brand_item_classNumber  city_item_classNumber  shop_user_classNumber  \
0                    65.0                    3.0                  119.0   
1                    65.0                    3.0                  119.0   
2   

In [55]:
train_drop_fea = ['shop_review_positive_rate_ave', 'shop_score_service_ave', 'shop_score_delivery_ave', 'shop_score_description_ave']
test_drop_fea = ['shop_review_positive_rate_ave', 'shop_score_service_ave', 'shop_score_delivery_ave', 'shop_score_description_ave']
def dropFeture(drop_fea, df):
    for fea in drop_fea:
        df = df.drop(fea, 1)
    return df
train_df = dropFeture(train_drop_fea, train_df)
test_df = dropFeture(test_drop_fea, test_df)


In [56]:
print(train_df.columns.values)
print(test_df.columns.values)

['instance_id' 'item_id' 'item_category_list' 'item_property_list'
 'item_brand_id' 'item_city_id' 'item_price_level' 'item_sales_level'
 'item_collected_level' 'item_pv_level' 'user_id' 'user_gender_id'
 'user_age_level' 'user_occupation_id' 'user_star_level' 'context_id'
 'context_timestamp' 'context_page_id' 'predict_category_property'
 'shop_id' 'shop_review_num_level' 'shop_review_positive_rate'
 'shop_star_level' 'shop_score_service' 'shop_score_delivery'
 'shop_score_description' 'is_trade' 'date' 'weekday' 'day' 'hour'
 'context_page_id_1' 'context_page_id_2' 'context_page_id_3'
 'context_page_id_4' 'item_price_level_0' 'item_price_level_1'
 'item_pv_level_low' 'shop_review_num_level_lowHigh'
 'shop_star_level_lowHigh' 'shop_star_level_middle' 'shop_star_level_loss'
 'day_hour' 'predict_category_list' 'predict_category_set'
 'real_item_category_list' 'predict_property_list'
 'match_category_proportion' 'match_property_proportion'
 'predict_category_number' 'predict_property_num

In [57]:
# #添加one-hot编码并保留原字段
# def addOneHot(df, colName):
#     colDum = pd.get_dummies(df[colName], prefix=colName)
#     df = pd.concat([df, colDum], axis=1)
#     return df

# #将相关字段进行one-hot编码
# def oneHotData(train_df, test_df, colSet):
#     for colName in colSet:
#         train_df = addOneHot(train_df, colName)
#         test_df = addOneHot(test_df, colName)
#     return train_df, test_df

# oneHotColSet = ['item_sales_level', 'item_collected_level', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']
# train_df, test_df = oneHotData(train_df, test_df, oneHotColSet)

# print(train_df.head(10))
# print(test_df.head(10))

In [58]:
print(len(test_df_b.columns.values))
print(len(test_df.columns.values))

test_df = pd.merge(test_df_b[['instance_id']], test_df, on=['instance_id'])

print(len(test_df_b))
print(len(test_df))
print(len(test_df.columns.values))


26
116
42888
42888
116


In [59]:
print(test_df.columns.values)
print(test_df_b.columns.values)

['instance_id' 'item_id' 'item_category_list' 'item_property_list'
 'item_brand_id' 'item_city_id' 'item_price_level' 'item_sales_level'
 'item_collected_level' 'item_pv_level' 'user_id' 'user_gender_id'
 'user_age_level' 'user_occupation_id' 'user_star_level' 'context_id'
 'context_timestamp' 'context_page_id' 'predict_category_property'
 'shop_id' 'shop_review_num_level' 'shop_review_positive_rate'
 'shop_star_level' 'shop_score_service' 'shop_score_delivery'
 'shop_score_description' 'date' 'weekday' 'day' 'hour' 'day_hour'
 'context_page_id_1' 'context_page_id_2' 'context_page_id_3'
 'context_page_id_4' 'item_price_level_0' 'item_price_level_1'
 'item_pv_level_low' 'shop_review_num_level_lowHigh'
 'shop_star_level_lowHigh' 'shop_star_level_middle' 'shop_star_level_loss'
 'predict_category_list' 'predict_category_set' 'real_item_category_list'
 'predict_property_list' 'match_category_proportion'
 'match_property_proportion' 'predict_category_number'
 'predict_property_number' 'isFir

In [60]:
# 导出训练集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)
    
exportResult(train_df, 'chusai_train_df')
exportResult(test_df, 'chusai_test_df')


In [61]:
print(len(test_df))

42888
